# Tarea 2

### Integrantes

---



*   Arath Alejandro Reyes López
*   Jesús Eduardo García Tapia
*   Valeria Lucero Cruz Estrada
*   José Antonio Ramirez Maciel
*   Juan Manuel Hernández Acosta

## Preeliminares

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"
!git clone "https://github.com/ArathReyes/Risk-Management/"

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 823, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 823 (delta 36), reused 0 (delta 0), pack-reused 768
Receiving objects: 100% (823/823), 41.20 MiB | 36.84 MiB/s, done.
Resolving deltas: 100% (541/541), done.
fatal: destination path 'Risk-Management' already exists and is not an empty directory.


In [ ]:
%%R
#Instalación de paqueterías
install.packages("quantmod")
install.packages("data.table")
install.packages("PerformanceAnalytics")
install.packages("Deriv")
library(Deriv)
library(quantmod)
library(data.table)
library("PerformanceAnalytics")
library(dplyr)

con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘zoo’, ‘TTR’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[

In [ ]:
%%R
## Funciones a utilizar

# Interpolación alambrada
talamb=function(nodos,curva,plazos){
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n){
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

# Diagonalización
diagv=function(x)	{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax){
    res[i,i]=x[i]
  }
  res
}

# Cuantil
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

In [ ]:
%%R
#VaR con alisado

wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 

## Parámetros

In [ ]:
%%R
fval=as.Date("20220331",format="%Y%m%d") #Fecha de valoración
alpha = 0.98
itpl = 0 # 0 Interpolación Lineal, 1 Interpolación Alambrada

### Ejercicio 1
1000 acciones de Grupo Carso, -5000 de América Móvil y 1200 WalMart.

In [ ]:
%%R
Symbols<-c  ("AMXL.MX", "GCARSOA1.MX", "WALMEX.MX" )
pos_eq=c(-5000,1000,1200)
nh=3660 # Días de historia

### Ejercicio 2
1500 Dólares (peso dólar), 700 Euros, y -600 Libras Esterlinas.

In [ ]:
%%R
SymbolsFX<-c("EURUSD=X","GBPUSD=X","USDMXN=X")
pos_fx=c(700,-600,1500)
nh=3660 # Días de historia

### Ejercicio 3
2 bonos largos, uno cupón cero de CETES (con la curva guber) con nocional de 1500 vencimiento de 180 días, un Bono M con tasa fija de 6.5% con un cupon de 6.05% anual vencimiento de 3600 días y nocional de 1000, un bono corto 1000 Bondes con plazo 707 con cupón que paga 28 días (calcular cupón basado en curva de fondeo y descontar con la curva correspondiente).

In [ ]:
%%R

#CETES
base = "Risk-Management/data/tasa_guber.txt"
plazos_bcc = cbind(180)
contratos_bcc = cbind(1500)
nominal_bcc = 10


#BONO M CON TASA FIJA

btasadesc="Risk-Management/data/tasa_yield.txt"
tf = cbind(0.065)
tfcupon_bm = cbind(0.0605)
plazos_bm = cbind(3600)
plazocupon_bm = cbind(182)
contratos_bm = cbind(1000)
nominal_bm = 100

#BONDES D
btasadescst="Risk-Management/data/tasa_guber_st.txt"
btasafondeo="Risk-Management/data/tfondeo.txt"
plazos_bdm= cbind(707, 707) #Vencimiento del bono
plazocupon_bdm= cbind(28, 28) #plazos_bdm fijos de cada cupón
contratos_bdm = cbind(-1000, 0)
nominal_bdm=100

### Ejercicio 4
100 contratos de compra de futuros de peso dólar con un strike de 20.83 vencimiento de 5 días, y 50 contratos
de venta de futuros del IPC con strike de 49525 vencimiento de 53 días.

In [ ]:
%%R
#FORWARDS TDC
bext="Risk-Management/data/tasa_libor.txt"
bdom="Risk-Management/data/tasa_fwd.txt"
SymbolsFX_ftdc<-c("USDMXN=X", "GBPUSD=X" ) 
plazos_fwd=cbind(5)
contratos_fwd=cbind(100)
kst_fwd=cbind(20.83)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.

#FORWARDS DE IPC
#Descontamos con gubernamental
base="Risk-Management/data/tasa_guber.txt"
SymbolsEQ_find<-c("^MXX", "GCARSOA1.MX" )
plazos_fwd_ind=cbind(53)
contratos_fwd_ind=cbind(50)
kst_fwd_ind=cbind(49525)
nominal_fwd_ind=1

### Ejercicio 5

Un swap largo nocional de 16 millones pagando tasa fija de 6.6% anual (cada 28 días) y recibiendo tasa flotante de la curva TIIE vencimiento de 588 días, un swap corto con nocional de 12 millones pagando tasa variable de la TIIE y recibiendo fija de 5.9% vencimiento de 270.

In [ ]:
%%R
btasadesc_sw="Risk-Management/data/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="Risk-Management/data/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.066,0.059) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(16000000,-12000000) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1, 1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,1) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

### Ejercicio 6
Dos opciones europeas, una de tasa de interés larga call con strike de 5.8 % vencimiento de 1700 días 1000
contratos, y otra put larga de tasa de interés con strike de 6.0 % de la TIIE nocional de 500 contratos vencimiento
700 días. (se valuará basado en las superficies de volatilidad si es in o out the money y en las curvas de tasa de interés TIIE o Pagarés).

In [ ]:
%%R
btasadesc_oir="Risk-Management/data/tasa_TIIE_SW_OP.txt" #Tasa de descuento
btasaspot_oir="Risk-Management/data/tasa_DIRS_SW_OP.txt" #Tasa Spot
bvolspot_oir="Risk-Management/data/tvoltiie_opc.txt" #Volatilidad
plazos_oir=cbind(1700, 700) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor) 
K_oir=cbind(0.058,0.06) #Strike
contratos_oir=cbind(1000,500)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple

## Carga de Datos

### Ejercicio 1

In [ ]:
%%R
start_date=Sys.Date()-nh #Fecha inicial
dataEnv<-new.env()
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)

bt.prep(dataEnv,align='remove.na',fill.gaps = T)
stock_prices = dataEnv$prices

###Ejercicio 2

In [ ]:
%%R
  start_date=Sys.Date()-nh #Fecha inicial
  dataEnvFX<-new.env() #Creación del objeto para guardar los datos
  getSymbols.yahoo(SymbolsFX,env=dataEnvFX,from=start_date) #Obtención de datos
  bt.prep(dataEnvFX,align='remove.na',fill.gaps=T) #Limpieza
  stock_pricesFX = dataEnvFX$prices #Obtenemos el precio

  stock_pricesFX=cbind(stock_pricesFX[,1,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,2,with=F]*stock_pricesFX[,3,with=F],stock_pricesFX[,3,with=F])
stock_prices_EQFX=merge(stock_pricesFX,stock_prices,join = "inner")

aux2=data.table(Date=as.Date(index(stock_prices_EQFX)),coredata(stock_prices_EQFX))


###Ejercicio 3

In [ ]:
%%R

# CETES
data_cetes<-read.table(base)
n_gov<-nrow(data_cetes)
m_gov=ncol(data_cetes)
x_orig_gov=data.frame(data_cetes[2:n_gov,1:m_gov])
x_orig_gov=as.data.table(mutate(data_cetes[2:n_gov,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data_cetes[1,2:m_gov])

# BONO M
data_m<-read.table(btasadesc)
n_bonom<-nrow(data_m)
m_bonom=ncol(data_m)
x_orig_bonom=data.frame(data_m[2:n_bonom,1:m_bonom])
x_orig_bonom=as.data.table(mutate(data_m[2:n_bonom,1:m_bonom],Date=as.Date(V1,format="%Y%m%d")))
x_orig_bonom=x_orig_bonom%>%select(-V1)
nodos_bonom=data.frame(data_m[1,2:m_bonom])

# BONDE D

data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
X_orig_bd=as.data.table(mutate(data1[2:n,1:m_bd],Date=as.Date(V1,format="%Y%m%d")))
X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
print(head(X1_orig[,1:4]))
nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])

X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #Sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

# Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# Rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 

# Buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.


          V1         V2         V3         V4
2 2022-03-31 0.07845614 0.07749243 0.07650381
3 2022-03-30 0.07663158 0.07749243 0.07650381
4 2022-03-29 0.07663158 0.07749243 0.07685555
5 2022-03-28 0.07753656 0.07749243 0.07685555
6 2022-03-25 0.07663158 0.07749243 0.07685555
7 2022-03-24 0.07663158 0.07742413 0.07685555


###Ejercicio 4

In [ ]:
%%R
#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))

x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)


###Para Dolar

if (yext==1)
{
#Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial


  
#Creación del objeto para guardar los datos
  dataEnvFX_ftdc<-new.env()
  
#obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX_ftdc,from=start_date, to=(fval))
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX_ftdc,align='remove.na',fill.gaps=T)
  
#muestra de datos
  head(dataEnvFX_ftdc$prices[,2])
  
#Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX_ftdc$prices[,2])),coredata(dataEnvFX_ftdc$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}


# CARGA DE DATOS DE  FORWARD DE IPC

data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])

  #Cargar los símbolos de yahoo finance para EQ
  start_date=fval-nh #fecha inicial
  
  #Creación del objeto para guardar los datos
  dataEnvEQ<-new.env()
  
  #obtener los datos
  getSymbols.yahoo(SymbolsEQ_find,env=dataEnvEQ,from=start_date, to=(fval))
  #limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvEQ,align='remove.na',fill.gaps=T)
  
  #muestra de datos
#  head(dataEnvEQ$prices)
  
  #Nos quedamos con los precios
  X3_find=data.table(Date=as.Date(index(dataEnvEQ$prices[,2])),coredata(dataEnvEQ$prices[,2]))

###Ejercicio 5

In [ ]:
%%R
data1<-read.table(btasadesc_sw)
n1<-nrow(data1)
m1_orig_sw=ncol(data1)
X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X1_orig_sw=X1_orig_sw%>%select(-V1)
nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
data2<-read.table(btasacupvar_sw)
n2<-nrow(data2)
m2_orig_sw=ncol(data2)
X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
X2_orig_sw=X2_orig_sw%>%select(-V1)
nodos2_sw=data.frame(data2[1,2:m2_orig_sw])

###Ejercicio 6

In [ ]:
%%R
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])

## Integración de Insumos

### Cruce de tablas

In [ ]:
%%R
#INTERSECCIÓN DE FECHAS DE TODOS LOS INSUMOS    
  

lin_gub=data.table(Date=as.Date(aux2[x_orig_gov,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield

#lin_gub_aux=data.table(Date=as.Date(lin_gub[x_orig_bonom,on=.(Date),nomatch=0]$Date)) 

lin_gub_bmybdst_aux=data.table(Date=as.Date(lin_gub[X3_orig_bd,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield y st (bonde)


lin_gub_bmybdst=data.table(Date=as.Date(X3_ftdc[lin_gub_bmybdst_aux,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield y st (bonde)

lin_gub_bmybdst_flib=data.table(Date=as.Date(lin_gub_bmybdst[x1_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield,  st (bonde), libor

lin_gub_bmybdst_flibfwd=data.table(Date=as.Date(lin_gub_bmybdst_flib[x2_ftdc,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd

lin_gub_bmybdst_flibfwdspind=data.table(Date=as.Date(lin_gub_bmybdst_flibfwd[X3_find,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index



lin_gub_bmybdst_flibfwdspind_swcup=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind[X1_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón

lin_gub_bmybdst_flibfwdspind_swcupvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcup[X2_orig_sw,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón

lin_gub_bmybdst_flibfwdspind_swcupvp_oirs=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp[x1_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirs[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp

lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, yield, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol


lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=unique(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)

n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
n

[1] 253


### Formato de historia

#### Historia de todos

In [ ]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
n

[1] 253


#### Ejercicios 1 y 2

In [ ]:
%%R
#historia de acciones y divisas
stock_prices_EQFX=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[aux2,on=.(Date),nomatch=0][order(-Date)]
stock_prices_EQFX=stock_prices_EQFX%>%select(-Date)

#### Ejercicio 3

In [ ]:
%%R
#Historia de curva gubernamental y yield
x_orig_gov=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x_orig_gov,on=.(Date),nomatch=0][order(-Date)]
x_orig_gov=x_orig_gov%>%select(-Date)

#Historia de curvas de bonde
X1_orig=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig,on=.(Date),nomatch=0][order(-Date)]
X1_orig=X1_orig%>%select(-Date)
X2_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X2_orig_bd=X2_orig_bd%>%select(-Date)
X3_orig_bd=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_orig_bd,on=.(Date),nomatch=0][order(-Date)]
X3_orig_bd=X3_orig_bd%>%select(-Date)



#### Ejercicio 4

In [ ]:
%%R
#historia de curvas de forward tdc
x1_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_ftdc,on=.(Date),nomatch=0][order(-Date)]
x1_ftdc=x1_ftdc%>%select(-Date)/100
x2_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_ftdc,on=.(Date),nomatch=0][order(-Date)]
x2_ftdc=x2_ftdc%>%select(-Date)/100
X3_ftdc=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_ftdc,on=.(Date),nomatch=0][order(-Date)]
X3_ftdc=X3_ftdc%>%select(-Date)

#historia de curvas de forward ind
#CONSIDERAR LA CURVA GUBERNAMENTAL X1_ORIG_GOV
X3_find=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X3_find,on=.(Date),nomatch=0][order(-Date)]
X3_find=X3_find%>%select(-Date)

#### Ejercicio 5

In [ ]:
%%R
X1_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X1_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X1_orig_sw=X1_orig_sw%>%select(-Date)/100
X2_orig_sw=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_sw,on=.(Date),nomatch=0][order(-Date)]
X2_orig_sw=X2_orig_sw%>%select(-Date)/100

#### Ejercicio 6

In [ ]:
%%R
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

## Valuación | Derivadas

### Acciones y divisas

In [ ]:
%%R
#divisas y acciones CÁLCULO

x0_acc_div=stock_prices_EQFX[1,]
DeltaX_acc_div=as.matrix(log(as.matrix(stock_prices_EQFX[1:(n-1)])/as.matrix(stock_prices_EQFX[2:(n)])))
V0_acc_div=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
mu_acc_div=0
g_acc_div=cbind(t(pos_fx),t(pos_eq))
g_acc_divt=cbind(t(pos_fx),t(pos_eq))*x0_acc_div
gamma_acc_divt=matrix(0,ncol(x0_acc_div),ncol(x0_acc_div))


m_fx=length(pos_fx)
m_acc=length(pos_eq)

### CETES

In [ ]:
%%R

#CETE CÁLCULO
#paso para interpolar las tasas
m=ncol(plazos_bcc)
x_bcc=matrix(0,n,m)
for (i in 1:(n))
{
  #x[i,]=approx(nodos_gov,x_orig_gov[i,],plazos_bcc)$y
  x_bcc[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],plazos_bcc,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],plazos_bcc)}
}

x0_bcc=x_bcc[1,]

#función
bonocupcero = function(i,t)
{
  1/(1+i*t/360)
}

#funcion primera derivada con tiempo
bcc_dt = function(i,t)
{
  -(i/360)/(1+i*t/360)^2
}

#funcion primera derivada con tasa interés
bcc_di = function(i,t)
{
  -(t/360)/(1+i*t/360)^2
}

#funcion segunda derivada con tasa de interés
bcc_di2 = function(i,t)
{
  2*(t/360)^2/(1+i*t/360)^3
}

V0_bcc=as.matrix(bonocupcero(x0_bcc,plazos_bcc))*contratos_bcc*nominal_bcc #Valor actual de cada bono

VT0_bcc=sum(V0_bcc) #Valor total del portafolio al tiempo 0

mu_bcc=bcc_dt(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
g_bcc=bcc_di(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
g_bcct=g_bcc*x0_bcc
gamma_bcc=bcc_di2(x0_bcc,plazos_bcc)*contratos_bcc*nominal_bcc
gamma_bcct2=diagv(gamma_bcc*(x0_bcc)^2)



print("Valor actual")
print(VT0_bcc)

[1] "Valor actual"
[1] 14426.87


### Bono M

In [ ]:
%%R
#BONO M  CÁLCULO
#Posición inicial
#interpolación de tasas y volatilidades
m=ncol(plazos_bm)
X_bm=matrix(0,n,m)
for (i in 1:(n))
{
  X_bm[i,]=if(itpl==0){approx(nodos_bonom,x_orig_gov[i,],plazos_bm,rule=2)$y}else{talamb(nodos_bonom,x_orig_gov[i,],plazos_bm)}
}


#Función de valoración por tasa yield
bonoMyield=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
{
  N=as.integer(plazos_bm/plazocupon_bm)+1   
  p1=plazos_bm-plazocupon_bm*(N-1)   
  a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
  ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)   
}

x0_bm=0.045 #tasas de descuento valor actual
V0_bm=bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm)

#función para derivar con respecto al tiempo a la función de Bono General
bonoMyieldm=function(x, plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm) #valoración bono tasa fija
{
  N=plazos_bm/plazocupon_bm+1   
  p1=plazos_bm-plazocupon_bm*(N-1)   
  a=(1-(1+x*plazocupon_bm/360)^(-N))/(plazocupon_bm*x/360)
  ((contratos_bm*nominal_bm*tfcupon_bm*plazocupon_bm/360)*a+(contratos_bm*nominal_bm)/((1+x*plazocupon_bm/360)^N))*(1+x*plazocupon_bm/360)^(1-p1/plazocupon_bm)   
}

mu_bm=eval(Deriv(bonoMyieldm(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "plazos_bm"))
mu_bm[1,1]
g_bm=eval(Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm"))
g_bmt=g_bm*x0_bm
gamma_bm=eval(Deriv(bonoMyield(x0_bm,plazos_bm, plazocupon_bm, tfcupon_bm, nominal_bm, contratos_bm), "x0_bm",nderiv=2))
gamma_bmt2=diagv(gamma_bm*(x0_bm)^2)
gamma_bmt2[is.nan(gamma_bmt2)] <- 0 #quitamos NaN
gamma_bmt2[is.na(gamma_bmt2)] <- 0 #quitamos Na
gamma_bmt2[is.infinite(gamma_bmt2)] <- 0 

mu_bm[is.nan(mu_bm)] <- 0 #quitamos NaN
mu_bm[is.na(mu_bm)] <- 0 #quitamos Na
mu_bm[is.infinite(mu_bm)] <- 0

g_bmt[is.nan(g_bmt)] <- 0 #quitamos NaN
g_bmt[is.na(g_bmt)] <- 0 #quitamos Na
g_bmt[is.infinite(g_bmt)] <- 0


print("Valor actual")
print(V0_bm[1,1])



[1] "Valor actual"
[1] 113037


### Bonde D

In [ ]:
%%R
#BONDE D CÁLCULO
names(X2_orig_bd)[1] = 'Date'
X2_pr=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[X2_orig_bd, on = .(Date),nomatch=0][order(-Date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
m=ncol(plazos_bdm)


N_bd=as.integer(plazos_bdm/plazocupon_bdm)+1 #número de cupones a pagar
VTplazos_bdm=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdm de todos los contratos_bdm
contratos_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los contratos_bdm de todos los flujos de todos los contratos_bdm
nominal_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los nominal_bdmes de todos los flujos de todos los contratos_bdm
plazocupon_bdmT=matrix(0,1,sum(N_bd)) #vector de todos los plazos_bdmcupon de todos los flujos de todos los contratos_bdm
tasafijaT_bd=matrix(0,1,sum(N_bd)) #vector de tasas fijas de todos los flujos de todos los contratos_bdm
ulNomT_bd=matrix(0,1,sum(N_bd)) #vector de contratos_bdm a final de flujo


plazini_bd=plazos_bdm-plazocupon_bdm*(N_bd-1) #vector de plazos_bdm iniciales
ddv=plazocupon_bdm-plazini_bd #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_bdm[,1:sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,1:sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])
    plazocupon_bdmT[,1:sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
    tasafijaT_bd[,1]=tfcupon[j]
    tasafijaT_bd[,2:sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos_bdm[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazini_bd[j],plazos_bdm[j], by=plazocupon_bdm[j])
    contratos_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(contratos_bdm[j],contratos_bdm[j])					
    plazocupon_bdmT[,(sum(N_bd[1:j-1])+1):sum(N_bd[1:j])]=seq(plazocupon_bdm[j],plazocupon_bdm[j])
    tasafijaT_bd[,(sum(N_bd[1:j-1])+1)]=tfcupon[j]
    tasafijaT_bd[,(sum(N_bd[1:j-1])+2):sum(N_bd[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT_bd[,sum(N_bd[1:j])]=contratos_bdm[j]
  }
}

Xvp_bd=matrix(0,n,ncol(VTplazos_bdm))
Xst_bd=matrix(0,n,ncol(VTplazos_bdm))

for (i in (1:n))
{
  Xvp_bd[i,]=if(itpl==0){approx(nodos_gov,x_orig_gov[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[i,],VTplazos_bdm)}
  Xst_bd[i,]=if(itpl==0){approx(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm,rule=2)$y}else{talamb(nodos3_bd,X3_orig_bd[i,],VTplazos_bdm)}
  
}


bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  tasafijaT=matrix(0,1,sum(N))
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


V0_bdm=bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)



d_bondeD_t=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  
  V0=matrix(0,1,count(N))
  V0f=-((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((Xvp+Xst)/360)
  for (j in (1:count(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 


#la derivada con respecto al valor presente y a la sobretasa es igual
d_bondeD_x=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  V0f=-((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360)
  V0f
} 


d_bondeD_tc=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }
  
  V0f=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)))*nominal_bdm
  V0f
}



mgamma_bondeD=function(contratos_bdmT,  nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp, Xst, N,ddv)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos
{
  tasafijaT=matrix(0,1,sum(N))
  tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
  tfcupondev=matrix(0,1,m) #cupón de los días devengados
  tfcupgen=((1+tf_act/360)^(plazocupon_bdm[1])-1)*360/plazocupon_bdm[1] #el segundo al último cupón de todos los bonos
  #calcula cupones de bonos
  for (j in (1:m))
  {
    tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
    tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon_bdm[1]-ddv[j])-1)*360/plazocupon_bdm[1]
  }
  
  
  for (j in (1:m))
  {
    if (j==1)
    {
      tasafijaT[,1]=tfcupon[j]
      tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
    else
    {
      tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
      tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    }
  }

  c11=matrix(0,1,length(contratos_bdmT)) #segunda derivada del cupón
  c12=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360) #primera del cupón y primera del vp
  c13=c12 #primera del cupón y primera del st
  c21=-((((contratos_bdmT*(plazocupon_bdmT/360)))/(1+(Xvp+Xst)*VTplazos_bdm/360)^2))*nominal_bdm*((VTplazos_bdm)/360) #primera del vp y primera del cupon
  c22=2*((((contratos_bdmT*(tasafijaT)*(plazocupon_bdmT/360))+ulNomT_bd)/(1+(Xvp+Xst)*VTplazos_bdm/360)^3))*nominal_bdm*((VTplazos_bdm)/360)^2 #segunda del vp
  c23=c22 #primera del vp y primera del st
  c31=c12 #primera del st y primera del cupon
  c32=c22 #primera del vp y primera del st
  c33=c22 #segunda del st
  rbind(cbind(diagv(c11),diagv(c12), diagv(c13)),cbind(diagv(c21),diagv(c22), diagv(c23)),cbind(diagv(c31),diagv(c32), diagv(c33)))
}


#mgamma_bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)

X_bd_tc=matrix(1,n,ncol(contratos_bdmT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp_bd),as.matrix(Xst_bd))
mu_bd=d_bondeD_t(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
g_bd=cbind(d_bondeD_tc(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv),d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv),d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv))
x0_bd_ext=cbind(matrix(tf_act,1,ncol(contratos_bdmT)),t(as.matrix(Xvp_bd[1,])),t(as.matrix(Xst_bd[1,])))
g_bdt=g_bd*x0_bd_ext
gamma_bd=mgamma_bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
gamma_bdt2=gamma_bd*(t(x0_bd_ext)%*%x0_bd_ext)

print("Valor actual")
print(V0_bdm[1,1])


[1] "Valor actual"
[1] -94292.17


###Forwards de tipo de cambio


In [ ]:
%%R
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################
#FORWARDS Y/O FUTUROS DE TIPO DE CAMBIO CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd)
X1_fwtdc=matrix(0,n,m)
X2_fwtdc=matrix(0,n,m)

for (j in 1:n)
{
  X1_fwtdc[j,]=if(itpl==0){approx(nodos1_ftdc,x1_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwtdc[j,]=if(itpl==0){approx(nodos2_ftdc,x2_ftdc[j,],plazos_fwd,rule=2)$y}else{talamb(nodos2_ftdc,x2_ftdc[j,],plazos_fwd)}
  if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}


futuroTC = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f-k)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

X3_ftdc=as.matrix(X3_ftdc)

V0_fwtdc=futuroTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd


dfuturoTC_tl = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=-s*((1+tn*t/360)/(1+tl*t/360)^2)*(t/360) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_tn = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  f1=s*((1+tn*t/360)/(1+tl*t/360)) #Se obtiene el tipo de cambio forward
  t(as.numeric((f)/(1+t*tn/360)-(f1-k)/(1+t*tn/360)^2*(t/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_s = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  1/(1+t*tl/360) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
}

dfuturoTC_t = function(t,tl,tn,s,k) #t=dias por vencer, tn=tasa nacional para tipo de cambio forward, tl= tasa extranjera pra tipo de cambio forward, S=spot
{
  f=s*((1+tn/360)/(1+tl*t/360))-s*((1+tn*t/360)/(1+tl*t/360)^2)*(tl/360) #Se obtiene el tipo de cambio forward
  #t(as.numeric((f)/(1+t*tn/360)-(f-k)/(1+t*tn/360)^2*(tn/360))) #Se obtiene el valor del payoff a valor presente con el valor z que define si es largo o corto
  -(f-k)/(1+t*tn/360)^2*(tn/360)
}

mgamma_futTC= function(t,tl,tn,s,k,contratos_fwd,nominal_fwd)
{
c11= matrix(0,1,length(k))#spot spot
c12= -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd#spot tl
c13=  matrix(0,1,length(k))  #spot tn
c21=  -(((1+tn*t/360)/(1+tl*t/360)^2)*(t/360))/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl spot
c22= (2*s*((1+tn*t/360)/(1+tl*t/360)^3)*(t/360)^3)/(1+t*tn/360)*contratos_fwd*nominal_fwd #tl tl
c23= matrix(0,1,length(k)) #tl tn
c31=(((t/360)/(1+tl*t/360))/(1+t*tn/360)-(((1+tn*t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn spot
c32=   (-s*((t/360)^2/(1+tl*t/360)^2)/(1+t*tn/360)-(s*((t/360)/(1+tl*t/360)))/(1+t*tn/360)^2*(t/360))*contratos_fwd*nominal_fwd #tn tl
c33=  (s*((t/360)/(1+tl*t/360))/(1+t*tn/360)^2*(t/360)+2*(s*((1+tn*t/360)/(1+tl*t/360))-k)/(1+t*tn/360)^3*(t/360)^2-(s*((t/360)^2/(1+tl*t/360)))/(1+t*tn/360)^2)*contratos_fwd*nominal_fwd #tn tn
m=length(t)
rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}


X_ftdc=cbind(matrix(X3_ftdc,n,ncol(X1_fwtdc)),X1_fwtdc,X2_fwtdc) #Hist facto riesgo
xp_ftdc=t(as.matrix(X_ftdc[1,])) #Valores actuales X0
mu_ftdc=dfuturoTC_t(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd #primera derivada con respecto al tiempo evaluada en x0
g_ftdc=cbind(dfuturoTC_s(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tl(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd,dfuturoTC_tn(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd)*contratos_fwd*nominal_fwd) #gradiente evaluado en x0
g_ftdctT=xp_ftdc*g_ftdc #gradienete evaluado en x0 por x0
gamma_ftdc=mgamma_futTC(plazos_fwd,X1_fwtdc[1,],X2_fwtdc[1,],X3_ftdc[1,],kst_fwd,contratos_fwd,nominal_fwd) #evaluación de gamma
gamma_ftdctT2=gamma_ftdc*(t(xp_ftdc)%*%xp_ftdc) #evaluación de gamma multiplicado por x^{2}

print(V0_fwtdc)


          [,1]
[1,] -85.61414


###Forwards IPC

In [ ]:
%%R
#FORWARDS Y/O FUTUROS DE ÍNDICES CÁLCULO

################MATRICES DE INTERPOLACION LINEAL ####################

m=ncol(plazos_fwd_ind)
X1_fwind=matrix(0,n,m) #DIVIDENDOS
X2_fwind=matrix(0,n,m)

for (j in 1:n)
{
  #X1_fwind[j,]=if(itpl==0){approx(nodos1_,x1_ftdc[j,],plazos_fwd)$y}else{talamb(nodos1_ftdc,x1_ftdc[j,],plazos_fwd)}
  X2_fwind[j,]=if(itpl==0){approx(nodos_gov,x_orig_gov[j,],plazos_fwd_ind,rule=2)$y}else{talamb(nodos_gov,x_orig_gov[j,],plazos_fwd_ind)}
  #if(trlib==1){X1_fwtdc[j,]=((1+X1_fwtdc[j,])^(plazos_fwd/180)-1)*360/plazos_fwd} #transformación de act 180 a 360
}

X3_find=as.matrix(X3_find)

V0_fwind=futuroTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind

X_find=cbind(matrix(X3_find,n,ncol(X1_fwind)),X1_fwind,X2_fwind)
xp_find=t(as.matrix(X_find[1,]))
mu_find=dfuturoTC_t(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind
g_find=cbind(dfuturoTC_s(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tl(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind,dfuturoTC_tn(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind)*contratos_fwd_ind*nominal_fwd_ind)
g_findtT=xp_find*g_find
gamma_find=mgamma_futTC(plazos_fwd_ind,X1_fwind[1,],X2_fwind[1,],X3_find[1,],kst_fwd_ind,contratos_fwd_ind,nominal_fwd_ind)
gamma_findtT2=gamma_find*(t(xp_find)%*%xp_find)

print(V0_fwind)

         [,1]
[1,] 342308.9


### Swaps

In [ ]:
%%R
##SWAP TASA FIJA VS TASA VARIABLE CÁLCULO

##Interpolamos

nodosvp=nodos1_sw
nodostc=nodos2_sw
curvavp=as.matrix(X1_orig_sw)
curvatc=X2_orig_sw
n1=nrow(curvavp)
n2=nrow(curvatc)


m=max(ncol(plazos_sw),1) #número de contratos_sw swap a valorar
N=matrix(0,1,m) #es un vector de m valores donde se cargarán los m número de cupones a pagar para cada contrato
for (j in (1:m))
{
  N[j]=as.integer(plazos_sw[j]/plazocupon_sw[j])+1 #número de cupones a pagar
}
VTplazos_sw=matrix(0,1,sum(N)) #vector de todos los plazos_sw de todos los contratos_sw
contratos_swT=matrix(0,1,sum(N)) #vector de todos los contratos_sw de todos los flujos de todos los contratos_sw
nominal_swT=matrix(0,1,sum(N)) #vector de todos los nominal_swes de todos los flujos de todos los contratos_sw
por_swT=matrix(0,1,sum(N)) #vector de todos los dummy si paga o recibe de todos los flujos de todos los contratos_sw
plazocupon_swT=matrix(0,1,sum(N)) #vector de todos los plazos_swcupon de todos los flujos de todos los contratos_sw
tasafija_swT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos_sw
VTplazos_swc=matrix(0,1,sum(N)) #vector de todos los plazos_sw cortos de todos los contratos_sw


plazini=plazos_sw-plazocupon_sw*(N-1) #vector de plazos_sw iniciales

for (j in (1:m))
{
  if (j==1)
  {
    VTplazos_sw[,1:sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,1:sum(N[1:j])]=c(0,VTplazos_sw[,1:(sum(N[1:j])-1)])
    contratos_swT[,1:sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])
    nominal_swT[,1:sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,1:sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,1:sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,1:sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
  else
  {
    VTplazos_sw[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos_sw[j], by=plazocupon_sw[j])
    VTplazos_swc[,(sum(N[1:j-1])+1):sum(N[1:j])]=c(0,VTplazos_sw[,(sum(N[1:j-1])+1):(sum(N[1:j])-1)])
    contratos_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos_sw[j],contratos_sw[j])					
    nominal_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(nominal_sw[j],nominal_sw[j])
    por_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(por_sw[j],por_sw[j])
    plazocupon_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon_sw[j],plazocupon_sw[j])
    tasafija_swT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tasafija_sw[j],tasafija_sw[j])
  }
}


Xvp=matrix(0,n,ncol(VTplazos_sw))
Xtc=matrix(0,n,ncol(VTplazos_sw))
Xtcc=matrix(0,n,ncol(VTplazos_sw))
XtfwdT=matrix(0,n,ncol(VTplazos_sw))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodosvp,curvavp[i,],VTplazos_sw,rule=2)$y}else{talamb(nodosvp,curvavp[i,],VTplazos_sw)}
  Xtc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_sw,rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_sw)}
  Xtcc[i,]=if(itpl==0){approx(nodostc,curvatc[i,],VTplazos_swc, rule=2)$y}else{talamb(nodostc,curvatc[i,],VTplazos_swc)}
  
  XtfwdT[i,]=((1+Xtc[i,]*VTplazos_sw/360)/(1+Xtcc[i,]*VTplazos_swc/360)-1)*360/plazocupon_swT 
  for (j in (1:ncol(VTplazos_sw)))
  {
    if (VTplazos_sw[j]<= plazocupon_swT[j])
    {
      XtfwdT[i,j]=Xtc[i,j]
    }
    else
    {
      j=sum(N[1:j])
    }
  }
}


swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp, N)
{
  V0=matrix(0,1,ncol(N))
  V0f=(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT/360)))/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
  for (j in (1:ncol(N)))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
}

V0_sw=swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,], N)


#derivada del swap con respecto aL tiempo
dswap_dtt=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*Xvp/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto a la tasa cupón variable
dswap_dtc=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  (((contratos_swT*plazocupon_swT)/360)/(1+Xvp*VTplazos_sw/360))*nominal_swT*(-1)^por_swT
}

#derivada del swap con respecto al valor presente
dswap_dvp=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)
{
  -1*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
}

mgamma_swap=function(por_swT, contratos_swT, nominal_swT, XtfwdT, tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp)	#matriz de segundas derivadas parciales de la tasa valor presente y pago de flujos
{
  c11=matrix(0,1,ncol(contratos_swT))
  c21=-1*(((contratos_swT*(plazocupon_swT*VTplazos_sw/360^2)))/(1+Xvp*VTplazos_sw/360)^(2))*nominal_swT*(-1)^por_swT
  c12=-1*(((contratos_swT*plazocupon_swT*VTplazos_sw)/360^2)/(1+Xvp*VTplazos_sw/360)^2)*nominal_swT*(-1)^por_swT
  c22=2*(((contratos_swT*(XtfwdT-tasafija_swT)*(plazocupon_swT*VTplazos_sw^2/360^3)))/(1+Xvp*VTplazos_sw/360)^(3))*nominal_swT*(-1)^por_swT
  rbind(cbind(diagv(c11),diagv(c12)),cbind(diagv(c21),diagv(c22)))
}

X_sw=cbind( XtfwdT,Xvp)

mu_sw=sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))
g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]),dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,]))

#Multiplicación de la derivada por el punto x0
gt_sw=c( XtfwdT[1,],Xvp[1,])*g_sw

gamma_sw=mgamma_swap(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT, plazocupon_swT, VTplazos_sw, Xvp[1,])
tgamma_sw=c( XtfwdT[1,],Xvp[1,])%*%t(c( XtfwdT[1,],Xvp[1,]))*gamma_sw 

print("Valor por contrato")
print(V0_sw)
print("Valor de los swaps")
print(sum(V0_sw))


[1] "Valor por contrato"
         [,1]     [,2]
[1,] -20224.6 80734.57
[1] "Valor de los swaps"
[1] 60509.98


### Caps / Floors

In [ ]:
%%R


##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}


x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades



opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}


V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio


#pr_oirPRIMERA DERIVADA

dopctintt = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial europea y TIEMPO
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(( -(S*dnorm(d1)*vol)/(2*sqrt(t/365))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/365)*(-1)^cp_oir)*(exp(-vp*t/365)))}else{(( -(S*dnorm(d1)*vol)/(2*sqrt(t/360))+(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(t/360)*(-1)^cp_oir)*(1/(1+vp*t/360)^2))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)


dopctintS = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2
}

#dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintvol = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)

dopctintrho = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#derivada parcial call europea y rho
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  (if(cs_oir==1){-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)}else{-(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)


mgammaopc=function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)	#matriz de segundas derivadas parciales de la call: rho, spot y vol de spot
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}    
  c11=(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(t/365)^2}else{2*(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^3)*(-1)^cp_oir*(t/360)^2})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho con respecto a rho
  c21=(if(cs_oir==1){((pnorm(d1*(-1)^cp_oir))*(exp(-vp*t/365)))*(-t/365)}else{(pnorm(d1*(-1)^cp_oir)*(1/(1+vp*t/360)^2))*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)- (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir*(-t/365)}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360)^2)*(-1)^cp_oir*(-t/360)})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir #spot con respecto a rho
  c31=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol con respecto a rho
  c12=c21 #rho con respecto a spot
  c22=(if(cs_oir==1){(dnorm(d1)/(S*vol*sqrt(t/365)))*(exp(-vp*t/365))}else{(dnorm(d1)/(S*vol*sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) -2*((if(cs_oir==1){(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(exp(-vp*t/365)))*(-1)^cp_oir*(1/(S*vol*sqrt(t/365)))}else{(((pnorm(d1*(-1)^cp_oir))+( S*dnorm(d1*(-1)^cp_oir)-K_oir*dnorm(d2*(-1)^cp_oir)))*(1/(1+vp*t/360)))*(1/(S*vol*sqrt(t/360)))*(-1)^cp_oir})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2) +2*(if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)^3/(1+S*pr_oir/dct_oir)^3*contratos_oir*nominal_oir #spot spot 
  c32=(if(cs_oir==1){dnorm(d1)*d2/vol*(exp(-vp*t/365))}else{dnorm(d1)*d2/vol*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir) +   (if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360))})*(pr_oir/dct_oir)^2/(1+S*pr_oir/dct_oir)^2*contratos_oir*nominal_oir # vol spot
  c13=(if(cs_oir==1){(S*dnorm(d1)*(sqrt(t/365)))*(exp(-vp*t/365))*(-t/365)}else{(S*dnorm(d1)*(sqrt(t/360)))*(1/(1+vp*t/360)^2)*(-t/360)})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #rho vol
  c23=c32 #spot vol
  c33=(if(cs_oir==1){(dnorm(d1)*(S*sqrt(t/365)))*((d1*d2)/vol)*exp(-vp*t/365)}else{(dnorm(d1)*(S*sqrt(t/360)))*((d1*d2)/vol)*(1/(1+vp*t/360))})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)*contratos_oir*nominal_oir #vol vol
  # outm=matrix(0,3,length(plazos_oir)*3)
  m=length(t)
 rbind(cbind(diagv(c11),diagv(c12),diagv(c13)),cbind(diagv(c21),diagv(c22),diagv(c23)),cbind(diagv(c31),diagv(c32),diagv(c33)))
}

#mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)
X_oir=cbind(x1,x2,x3)
xp_oir=cbind(t(x01),t(x02),t(x03))
mu_oir=dopctintt(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir 
delta_oir_T=cbind(dopctintrho(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir, dopctintS(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir,dopctintvol(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir)
deltaT_oir_T=xp_oir*delta_oir_T
gammaT_oir=mgammaopc(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir,contratos_oir,nominal_oir)
gammaT_oirT2=gammaT_oir*(t(xp_oir)%*%xp_oir)

print(V0_oir)



         [,1]      [,2]
[1,] 1.002318 0.2479957


### Portafolio

In [ ]:
%%R
n_if=matrix(0,8,1)
n_if[1]=ncol(stock_prices_EQFX) #acciones y divisas
n_if[2]=ncol(x_bcc) #cetes
n_if[3]=ncol(X_bm) #bonos m
n_if[4]=ncol(X_bd_ext) #bonde
n_if[5]=ncol(X_ftdc) #fut tdc
n_if[6]=ncol(X_find) #fut ind
n_if[7]=ncol(X_sw) #swaps
n_if[8]=ncol(X_oir) #opciones tasa de interés

#valor del portafolios

V0_port=cbind(V0_bcc, V0_bm, V0_bdm, V0_fwtdc, V0_fwind, V0_sw, V0_oir)
V0T_port=sum(V0_port)



#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(stock_prices_EQFX,x_bcc,X_bm,X_bd_ext,X_ftdc,X_find,X_sw,X_oir) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

## Delta-Gamma

#### Acciones y divisas

##### Por factor de riesgo

In [ ]:
%%R
Delta_acc = DeltaX_acc_div[,c(4,5,6)]
Delta_div = DeltaX_acc_div[,c(1,2,3)]

mu_acc = 0
mu_div = 0

desvest_acc = apply(Delta_acc, 2, sd)
desvest_div = apply(Delta_div, 2, sd)

g_acct = g_acc_divt[,c(4,5,6)]
g_divt = g_acc_divt[,c(1,2,3)]

VarCovX_acc = cov(Delta_acc)
VarCovX_div = cov(Delta_div)

ajuste_cornish_acc_div = qnorm(alpha)

VaRfrDGN_acc = qnorm(1-alpha)*sqrt(g_acct*g_acct*desvest_acc*desvest_acc)
VaRfrDGN_div = qnorm(1-alpha)*sqrt(g_divt*g_divt*desvest_div*desvest_div)

CVaRfrDGN_acc = (dnorm(qnorm(1-alpha))/(1-alpha))*sqrt(g_acct*g_acct*desvest_acc*desvest_acc)*-1
CVaRfrDGN_div = (dnorm(qnorm(1-alpha))/(1-alpha))*sqrt(g_divt*g_divt*desvest_div*desvest_div)*-1

VaRfrCF_acc = ajuste_cornish_acc_div*sqrt(g_acct*g_acct*desvest_acc*desvest_acc)*-1
VaRfrCF_div = ajuste_cornish_acc_div*sqrt(g_divt*g_divt*desvest_div*desvest_div)*-1

CVaRfrCF_acc = (dnorm(qnorm(alpha))/(1-alpha))*sqrt(g_acct*g_acct*desvest_acc*desvest_acc)*-1
CVaRfrCF_div = (dnorm(qnorm(alpha))/(1-alpha))*sqrt(g_divt*g_divt*desvest_div*desvest_div)*-1

print('VaR por factor de riesgo Delta Gamma Normal acciones')
print(VaRfrDGN_acc)
print('CVaR por factor de riesgo Delta Gamma Normal acciones')
print(CVaRfrDGN_acc)
print('VaR por factor de riesgo Delta Gamma Normal divisas')
print(VaRfrDGN_div)
print('CVaR por factor de riesgo Delta Gamma Normal divisas')
print(CVaRfrDGN_div)

print('VaR por factor de riesgo Delta Gamma Normal CF acciones')
print(VaRfrCF_acc)
print('CVaR por factor de riesgo Delta Gamma Normal CF acciones')
print(CVaRfrCF_acc)
print('VaR por factor de riesgo Delta Gamma Normal CF divisas')
print(VaRfrCF_div)
print('CVaR por factor de riesgo Delta Gamma Normal CF divisas')
print(CVaRfrCF_div)

[1] "VaR por factor de riesgo Delta Gamma Normal acciones"
     AMXL.MX GCARSOA1.MX WALMEX.MX
1: -2864.347   -3468.902 -2978.942
[1] "CVaR por factor de riesgo Delta Gamma Normal acciones"
     AMXL.MX GCARSOA1.MX WALMEX.MX
1: -3376.419   -4089.053 -3511.501
[1] "VaR por factor de riesgo Delta Gamma Normal divisas"
    EURUSD.X  GBPUSD.X  USDMXN.X
1: -157.1863 -159.9017 -357.3584
[1] "CVaR por factor de riesgo Delta Gamma Normal divisas"
    EURUSD.X GBPUSD.X  USDMXN.X
1: -185.2872 -188.488 -421.2449
[1] "VaR por factor de riesgo Delta Gamma Normal CF acciones"
     AMXL.MX GCARSOA1.MX WALMEX.MX
1: -2864.347   -3468.902 -2978.942
[1] "CVaR por factor de riesgo Delta Gamma Normal CF acciones"
     AMXL.MX GCARSOA1.MX WALMEX.MX
1: -3376.419   -4089.053 -3511.501
[1] "VaR por factor de riesgo Delta Gamma Normal CF divisas"
    EURUSD.X  GBPUSD.X  USDMXN.X
1: -157.1863 -159.9017 -357.3584
[1] "CVaR por factor de riesgo Delta Gamma Normal CF divisas"
    EURUSD.X GBPUSD.X  USDMXN.X
1: -185.

##### Total

In [ ]:
%%R

VaRTotalDGN_acc = sqrt(as.matrix(g_acct)%*%VarCovX_acc%*%t(as.matrix(g_acct)))*qnorm(1-alpha)
VaRTotalDGN_div = sqrt(as.matrix(g_divt)%*%VarCovX_div%*%t(as.matrix(g_divt)))*qnorm(1-alpha)

CVaRTotalDGN_acc = sqrt(as.matrix(g_acct)%*%VarCovX_acc%*%t(as.matrix(g_acct)))*dnorm(qnorm(1-alpha))/(1-alpha)*-1
CVaRTotalDGN_div = sqrt(as.matrix(g_divt)%*%VarCovX_div%*%t(as.matrix(g_divt)))*dnorm(qnorm(1-alpha))/(1-alpha)*-1

VaRTotalCF_acc = ajuste_cornish_acc_div*sqrt(as.matrix(g_acct)%*%VarCovX_acc%*%t(as.matrix(g_acct)))*-1
VaRTotalCF_div = ajuste_cornish_acc_div*sqrt(as.matrix(g_divt)%*%VarCovX_div%*%t(as.matrix(g_divt)))*-1

CVaRTotalCF_acc = (dnorm(qnorm(alpha))/(1-alpha))*sqrt(as.matrix(g_acct)%*%VarCovX_acc%*%t(as.matrix(g_acct)))*-1
CVaRTotalCF_div = (dnorm(qnorm(alpha))/(1-alpha))*sqrt(as.matrix(g_divt)%*%VarCovX_div%*%t(as.matrix(g_divt)))*-1


print('VaR total Delta Gamma Normal acciones')
print(VaRTotalDGN_acc)
print('CVaR total Delta Gamma Normal acciones')
print(CVaRTotalDGN_acc)
print('VaR total Delta Gamma Normal divisas')
print(VaRTotalDGN_div)
print('CVaR total Delta Gamma Normal divisas')
print(CVaRTotalDGN_div)

print('VaR total Delta Gamma Normal CF acciones')
print(VaRTotalCF_acc)
print('CVaR total Delta Gamma Normal CF acciones')
print(CVaRTotalCF_acc)
print('VaR total Delta Gamma Normal CF divisas')
print(VaRTotalCF_div)
print('CVaR total Delta Gamma Normal CF divisas')
print(CVaRTotalCF_div)

[1] "VaR total Delta Gamma Normal acciones"
          [,1]
[1,] -4824.416
[1] "CVaR total Delta Gamma Normal acciones"
          [,1]
[1,] -5686.899
[1] "VaR total Delta Gamma Normal divisas"
          [,1]
[1,] -376.8845
[1] "CVaR total Delta Gamma Normal divisas"
          [,1]
[1,] -444.2619
[1] "VaR total Delta Gamma Normal CF acciones"
          [,1]
[1,] -4824.416
[1] "CVaR total Delta Gamma Normal CF acciones"
          [,1]
[1,] -5686.899
[1] "VaR total Delta Gamma Normal CF divisas"
          [,1]
[1,] -376.8845
[1] "CVaR total Delta Gamma Normal CF divisas"
          [,1]
[1,] -444.2619


###CETES

In [ ]:
%%R

Delta_bcc = x_bcc[1:(n-1),1]/x_bcc[2:(n),1]-1

desvest_bcc = sd(Delta_bcc)
sigma_gamma_bcc = sqrt(0.5*(gamma_bcct2[1,1]*desvest_bcc^2)^2+g_bcct[1,1]*desvest_bcc^2*g_bcct[1,1])
mu_3_bcc = (1/2 * (gamma_bcct2[1,1]*desvest_bcc^2)^3 + 3 * gamma_bcct2[1,1] * g_bcct[1,1] * desvest_bcc^2 * g_bcct[1,1])/sigma_gamma_bcc^3
ajuste_cornish_bcc = qnorm(1-alpha) + 1/6 * (qnorm(1-alpha)^2-1)*mu_3_bcc
aux_bcc = pnorm(-ajuste_cornish_bcc)

#Calculamos el VaR y CVaR por Cornish Fisher
VaRContCF_bcc = (mu_bcc[1,1] + sigma_gamma_bcc * ajuste_cornish_bcc * -1)
print("VaR total Delta Gamma CF")
print(VaRContCF_bcc)
CVaRContCF_bcc = (mu_bcc[1,1] + (dnorm(-ajuste_cornish_bcc)/aux_bcc)*sigma_gamma_bcc*-1)
print("CVaR total Delta Gamma CF")
print(CVaRContCF_bcc)

#Calculamos el VaR y CVaR por Delta Gamma Normal
VaRContDGM_bcc = (mu_bcc[1,1] + sigma_gamma_bcc * qnorm(1-alpha))
print("VaR total Delta Gamma Normal")
print(VaRContDGM_bcc)
CVaRContDGM_bcc = (mu_bcc[1,1] + (dnorm(qnorm(alpha))/(1-alpha))*sigma_gamma_bcc*-1)
print("CVaR total Delta Gamma Normal")
print(CVaRContDGM_bcc) 

[1] "VaR total Delta Gamma CF"
[1] -63.71016
[1] "CVaR total Delta Gamma CF"
[1] -63.90746
[1] "VaR total Delta Gamma Normal"
[1] -10.18972
[1] "CVaR total Delta Gamma Normal"
[1] -11.4639


### Bonos M
Observe que nuestra exposición al riesgo de mercado es nula, esto se debe al hecho de que la tasa bajo la cual está pactada el Bono M es fija, esto implica que no se verá afectada por las fluctuaciones de mercado. En efecto de lo anterior, el riesgo de mercado asumido presentado por el VaR y CVaR es cero. Sin embargo, no podemos afirmar en lo general que nuestra exposición al riesgo sea nula, dado que si bien el riesgo de mercado se nulifica al dejar la tasa fija, no podemos decir lo mismo del riesgo de crédito.

In [ ]:
%%R
gamma_bcct2

         [,1]
[1,] 42.12399


### Bondes D

#### Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [ ]:
%%R

Delta_X_bd_ext=X_bd_ext[1:(n-1),]/X_bd_ext[2:(n),]-1

##POR INSTRUMENTO

#Para obtener el VaR por contrato
VaRContDGM_bd=matrix(0,1,count(N_bd))
CVaRContDGM_bd=matrix(0,1,count(N_bd))
VaRContCF_bd=matrix(0,1,count(N_bd))
CVaRContCF_bd=matrix(0,1,count(N_bd))
#Para obtener el VaR por contrato por factor de riesgo
VaRContDGM_bdf=matrix(0,3,count(N_bd))
CVaRContDGM_bdf=matrix(0,3,count(N_bd))
VaRContCF_bdf=matrix(0,3,count(N_bd))
CVaRContCF_bdf=matrix(0,3,count(N_bd))

#V0=swap(porT, contratosT, nominalT, XtfwdT[1,], tasafijaT, plazocuponT, VTplazos, Xvp[1,],N)


#Valor de las derivadas evaluadas en el punto x0.
V0f_dtc_bd=d_bondeD_tc(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0f_dvp_bd=d_bondeD_x(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv)
V0f_dst_bd=V0f_dvp_bd

for (j in (1:count(N_bd)))
{
  if(j==1)
  { 
    x0cont_bd=c(X_bd_tc[1,1:sum(N_bd[1:j])],Xvp_bd[1,1:sum(N_bd[1:j])],Xst_bd[1,1:sum(N_bd[1:j])])
    #MATRIZ de factores de riesgo primera fila cupón, segunda valor presente, tercera sobretasa
    x0cont_bdf=rbind(c(X_bd_tc[1,1:sum(N_bd[1:j])],Xvp_bd[1,1:sum(N_bd[1:j])]*0,Xst_bd[1,1:sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,1:sum(N_bd[1:j])]*0,Xvp_bd[1,1:sum(N_bd[1:j])],Xst_bd[1,1:sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,1:sum(N_bd[1:j])]*0,Xvp_bd[1,1:sum(N_bd[1:j])]*0,Xst_bd[1,1:sum(N_bd[1:j])]))
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[1:sum(N_bd[1:j])],V0f_dvp_bd[1:sum(N_bd[1:j])],V0f_dst_bd[1:sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(X_bd_tc[,1:sum(N_bd[1:j])],Xvp_bd[,1:sum(N_bd[1:j])],Xst_bd[,1:sum(N_bd[1:j])]))*(n-1)/(n-2)	
    gamma_cont_bd=gamma_bd[1:(3*sum(N_bd[1:j])),1:(3*sum(N_bd[1:j]))]
    tgamma_bd= x0cont_bd%*%t(x0cont_bd)*gamma_cont_bd #gamma tilde
  }
  else
  {
    x0cont_bd=c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])])
    x0cont_bdf=rbind(c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0),
                     c(X_bd_tc[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xvp_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]*0,Xst_bd[1,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dvp_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dst_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(X_bd_tc[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xvp_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],Xst_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))*(n-1)/(n-2)
    gamma_cont_bd=gamma_bd[(3*sum(N_bd[1:j-1])+1):(3*sum(N_bd[1:j])),(3*sum(N_bd[1:j-1])+1):(3*sum(N_bd[1:j]))]
    tgamma_bd= x0cont_bd%*%t(x0cont_bd)*gamma_cont_bd #gamma tilde
  }
  
  
  tgammavar_bd=(tgamma_bd%*%VarCovXcont_bd)
  tgammavar2_bd=(tgamma_bd%*%VarCovXcont_bd)%*%(tgamma_bd%*%VarCovXcont_bd)
  VaRContDGM_bd[j]=ifelse((sum(diag(tgammavar_bd))/2+mucont_bd)>0,0,(sum(diag(tgammavar_bd))/2+mucont_bd))+(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*qnorm(1-alpha) #VaR Normal
  CVaRContDGM_bd[j]=ifelse((sum(diag(tgammavar_bd))/2+mucont_bd)>0,0,(sum(diag(tgammavar_bd))/2+mucont_bd))-(sqrt(sum(diag(tgammavar_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(qnorm(alpha))/(1-alpha)
  ###VaR Cornish Fisher
  tgammavar3_bd=tgammavar2_bd%*%(tgamma_bd%*%VarCovXcont_bd)
  asimPmargc_bd=sum(diag(tgammavar3_bd))+3*gtcont_bd%*%VarCovXcont_bd%*%tgamma_bd%*%VarCovXcont_bd%*%t(gtcont_bd) #gamma tercer momento
  asimPmargc_bd=asimPmargc_bd/(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))^3  #coef de asimetría
  VaRmargcstd_bd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bd  #VaR estandarizado
  VaRContCF_bd[j]=ifelse((sum(diag(tgammavar_bd))/2+mucont_bd)>0,0,(sum(diag(tgammavar_bd))/2+mucont_bd))+(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*VaRmargcstd_bd #VaR Normal
  CVaRContCF_bd[j]=ifelse((sum(diag(tgammavar_bd))/2+mucont_bd)>0,0,(sum(diag(tgammavar_bd))/2+mucont_bd))-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(VaRmargcstd_bd)/(pnorm(VaRmargcstd_bd))
  for (k in (1:3))
  {
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bdf[k,]))
    tgamma_bd= x0cont_bdf[k,]%*%t(x0cont_bdf[k,])*gamma_cont_bd #gamma tilde
    tgammavar_bd=(tgamma_bd%*%VarCovXcont_bd)
    tgammavar2_bd=(tgamma_bd%*%VarCovXcont_bd)%*%(tgamma_bd%*%VarCovXcont_bd)
    VaRContDGM_bdf[k,j]=(sum(diag(tgammavar_bd))/2+mucont_bd)-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*qnorm(alpha) #VaR Normal
    CVaRContDGM_bdf[k,j]=(sum(diag(tgammavar_bd))/2+mucont_bd)-(sqrt(sum(diag(tgammavar_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(qnorm(alpha))/(1-alpha)
    ###VaR Cornish Fisher
    tgammavar3_bd=tgammavar2_bd%*%(tgamma_bd%*%VarCovXcont_bd)
    asimPmargc_bd=sum(diag(tgammavar3_bd))+3*gtcont_bd%*%VarCovXcont_bd%*%tgamma_bd%*%VarCovXcont_bd%*%t(gtcont_bd) #gamma tercer momento
    asimPmargc_bd=asimPmargc_bd/(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))^3  #coef de asimetría
    VaRmargcstd_bd=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bd  #VaR estandarizado
    VaRContCF_bdf[k,j]=(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*VaRmargcstd_bd #VaR Normal
    CVaRContCF_bdf[k,j]=-(sqrt(sum(diag(tgammavar2_bd))/2+gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd)))*dnorm(VaRmargcstd_bd)/(pnorm(VaRmargcstd_bd))    
  }
  
}
print("VaR contrato Delta Gamma Normal:")
print(VaRContDGM_bd[1,1])
print("CVaR contrato Delta Gamma Normal:") 
print(CVaRContDGM_bd[1,1])
print("VaR contrato Delta Gamma CF:")
print(VaRContCF_bd[1,1])
print("CVaR contrato Delta Gamma CF:")
print(CVaRContCF_bd[1,1])
print("VaR contrato-factor de riesgo Delta Gamma Normal:")
print(VaRContDGM_bdf[1,1])
print("CVaR contrato-factor de riesgo Delta Gamma Normal:")
print(CVaRContDGM_bdf[1,1])
print("VaR contrato-factor de riesgo Delta Gamma CF:")
print(VaRContCF_bdf[1,1])
print("CVaR contrato-factor de riesgo Delta Gamma CF:")
print(CVaRContCF_bdf[1,1])

[1] "VaR contrato Delta Gamma Normal:"
[1] -175.5683
[1] "CVaR contrato Delta Gamma Normal:"
[1] -206.9554
[1] "VaR contrato Delta Gamma CF:"
[1] -175.5677
[1] "CVaR contrato Delta Gamma CF:"
[1] -206.9549
[1] "VaR contrato-factor de riesgo Delta Gamma Normal:"
[1] -144.5551
[1] "CVaR contrato-factor de riesgo Delta Gamma Normal:"
[1] -174.6483
[1] "VaR contrato-factor de riesgo Delta Gamma CF:"
[1] -168.3306
[1] "CVaR contrato-factor de riesgo Delta Gamma CF:"
[1] -198.4239


#### Riesgo a nivel total y total factor de riesgo

In [ ]:
%%R

VarCovX_bd_ext=cov(Delta_X_bd_ext)*(n-1)/(n-2)
tgammavar_bdt=(gamma_bdt2%*%VarCovX_bd_ext)
tgammavar2_bdt=(gamma_bdt2%*%VarCovX_bd_ext)%*%(gamma_bdt2%*%VarCovX_bd_ext) 
VaRTotalDGM=ifelse((sum(diag(tgammavar_bdt))/2+sum(mu_bd))>0,0,(sum(diag(tgammavar_bdt))/2+sum(mu_bd)))+(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*qnorm(1-alpha) #VaR Normal
CVaRTotalDGM=ifelse((sum(diag(tgammavar_bdt))/2+sum(mu_bd))>0,0,(sum(diag(tgammavar_bdt))/2+sum(mu_bd)))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(qnorm(alpha))/(1-alpha)

# VaR Cornish Fisher
tgammavar3_bdt=tgammavar2_bdt%*%(gamma_bdt2%*%VarCovX_bd_ext) 
asimPmargc_bdt=sum(diag(tgammavar3_bdt))+3*g_bdt%*%VarCovX_bd_ext%*%gamma_bdt2%*%VarCovX_bd_ext%*%t(g_bdt) #gamma tercer momento
asimPmargc_bdt=asimPmargc_bdt/(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))^3  #coef de asimetría
VaRmargcstd_bdt=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bdt  #VaR estandarizado
VaRTotalCF=ifelse((sum(diag(tgammavar_bdt))/2+sum(mu_bd))>0,0,(sum(diag(tgammavar_bdt))/2+sum(mu_bd)))+(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*VaRmargcstd_bdt #VaR Normal #VaR cf
CVaRTotalCF=ifelse((sum(diag(tgammavar_bdt))/2+sum(mu_bd))>0,0,(sum(diag(tgammavar_bdt))/2+sum(mu_bd)))-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(VaRmargcstd_bdt)/(pnorm(VaRmargcstd_bdt))

# Por factor de riesgo
VaRTotalDGMf=matrix(0,1,3)
CVaRTotalDGMf=matrix(0,1,3)
VaRTotalCFf=matrix(0,1,3)
CVaRTotalCFf=matrix(0,1,3)
 #MATRIZ de factores de riesgo primera fila cupón, segunda valor presente, tercera sobretasa
x0_bd_extf=rbind(cbind(matrix(tf_act,1,ncol(contratos_bdmT)),t(as.matrix(Xvp_bd[1,]))*0,t(as.matrix(Xst_bd[1,]))*0),
                 cbind(matrix(tf_act,1,ncol(contratos_bdmT))*0,t(as.matrix(Xvp_bd[1,])),t(as.matrix(Xst_bd[1,]))*0),
                 cbind(matrix(tf_act,1,ncol(contratos_bdmT))*0,t(as.matrix(Xvp_bd[1,]))*0,t(as.matrix(Xst_bd[1,]))))

for (j in (1:3))
{
g_bdt=g_bd*x0_bd_extf[j,]
gamma_bdt2=gamma_bd*((x0_bd_extf[j,])%*%t(x0_bd_extf[j,]))
VarCovX_bd_ext=cov(Delta_X_bd_ext)*(n-1)/(n-2)
tgammavar_bdt=(gamma_bdt2%*%VarCovX_bd_ext)
tgammavar2_bdt=(gamma_bdt2%*%VarCovX_bd_ext)%*%(gamma_bdt2%*%VarCovX_bd_ext) 
VaRTotalDGMf[,j]=(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*qnorm(1-alpha) #VaR Normal
CVaRTotalDGMf[,j]=-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(qnorm(alpha))/(1-alpha)
###VaR Cornish Fisher
tgammavar3_bdt=tgammavar2_bdt%*%(gamma_bdt2%*%VarCovX_bd_ext) 
asimPmargc_bdt=sum(diag(tgammavar3_bdt))+3*g_bdt%*%VarCovX_bd_ext%*%gamma_bdt2%*%VarCovX_bd_ext%*%t(g_bdt) #gamma tercer momento
asimPmargc_bdt=asimPmargc_bdt/(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))^3  #coef de asimetría
VaRmargcstd_bdt=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_bdt  #VaR estandarizado
VaRTotalCFf[,j]=(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*VaRmargcstd_bdt #VaR Normal #VaR cf
CVaRTotalCFf[,j]=-(sqrt(sum(diag(tgammavar2_bdt))/2+g_bdt%*%VarCovX_bd_ext%*%t(g_bdt)))*dnorm(VaRmargcstd_bdt)/(pnorm(VaRmargcstd_bdt))
}
sum(V0_bdm)
print("VaR total Delta Gamma Normal:")
print(VaRTotalDGM)
print("CVaR total Delta Gamma Normal:")
print(CVaRTotalDGM)
print("VaR total Delta Gamma CF:")
print(VaRTotalCF)
print("CVaR total Delta Gamma CF:")
print(CVaRTotalCF)
print("VaR total Delta Gamma Normal por Factor de Riesgo:")
print(VaRTotalDGMf)
print("CVaR total Delta Gamma Normal por Factor de Riesgo:")
print(CVaRTotalDGMf)
print("VaR total Delta Gamma CF por Factor de Riesgo:")
print(VaRTotalCFf)
print("CVaR total Delta Gamma CF por Factor de Riesgo:")
print(CVaRTotalCFf)

[1] "VaR total Delta Gamma Normal:"
          [,1]
[1,] -468.2529
[1] "CVaR total Delta Gamma Normal:"
          [,1]
[1,] -551.9645
[1] "VaR total Delta Gamma CF:"
          [,1]
[1,] -468.4329
[1] "CVaR total Delta Gamma CF:"
          [,1]
[1,] -552.1246
[1] "VaR total Delta Gamma Normal por Factor de Riesgo:"
          [,1]      [,2]      [,3]
[1,] -367.5774 -220.7697 -79.80457
[1] "CVaR total Delta Gamma Normal por Factor de Riesgo:"
          [,1]      [,2]      [,3]
[1,] -433.2909 -260.2377 -94.07159
[1] "VaR total Delta Gamma CF por Factor de Riesgo:"
          [,1]     [,2]      [,3]
[1,] -367.5774 -221.197 -79.86047
[1] "CVaR total Delta Gamma CF por Factor de Riesgo:"
          [,1]      [,2]      [,3]
[1,] -433.2909 -260.6175 -94.12128


###Forwards de Tipo de Cambio


In [ ]:
%%R
DeltaX_ftdc=DeltaX_port[,(sum(n_if[1:4],1)):(sum(n_if[1:5]))]


####Riesgo a Nivel Contrato y Nivel Contrato-Factor de Riesgo

In [ ]:
%%R


#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_ftdc=DeltaX_port[,(sum(n_if[1:4],1)):(sum(n_if[1:5]))] 
m_ftdc=ncol(plazos_fwd)
m=ncol(plazos_fwd)
VaRmargc_ftdc=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_ftdc=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_ftdc=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_ftdc=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_ftdc=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_ftdc=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_ftdc=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_ftdc=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_ftdc=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_ftdc=matrix(0,3,m) #CVar por contrato estandarizado
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)]) 
  tdelta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]   #delta tilde
  gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)  #paso clave
  tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
  Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
  tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
  mudg_ftdc=ifelse((sum(diag(tgammavar_ftdc))/2+mu_ftdc[i])>0,0,(sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]))
  desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))

  VaRmargc_ftdc[i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
  asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
  asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
  VaRmargcstd_ftdc[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
  VaRmargcCF_ftdc[i]=mudg_ftdc+desv_ftdc*VaRmargcstd_ftdc[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(VaRmargcstd_ftdc[i])/(pnorm(VaRmargcstd_ftdc[i])))

#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE *(1,0,0), (0,1,0), (0,0,1)
    #print(WFR[j,])
    tdelta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
    gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)*WFRM  #paso clave
   #print(WFRM)
    tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
    Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
    tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
    #mudg_ftdc=sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]
    mudg_ftdc=0
    desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))

    VaRmargc_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
    asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
    asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
    VaRmargcstd_fr_ftdc[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
    VaRmargcCF_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*VaRmargcstd_fr_ftdc[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(VaRmargcstd_fr_ftdc[j,i])/(pnorm(VaRmargcstd_fr_ftdc[j,i])))
  }



}

print("Valor a nivel contrato")
print(V0_fwtdc)
print("VaR a nivel contrato")
print(VaRmargc_ftdc)
print("VaR a nivel contrato CF")
print(VaRmargcCF_ftdc)
print("CVaR a nivel contrato")
print(CVaRmargc_ftdc)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_ftdc)


print("VaR a nivel contrato")
print(VaRmargc_fr_ftdc)
print("VaR a nivel contrato CF")
print(VaRmargcCF_fr_ftdc)
print("CVaR a nivel contrato")
print(CVaRmargc_fr_ftdc)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_fr_ftdc)

[1] "Valor a nivel contrato"
          [,1]
[1,] -85.61414
[1] "VaR a nivel contrato"
          [,1]
[1,] -23.85918
[1] "VaR a nivel contrato CF"
          [,1]
[1,] -23.85918
[1] "CVaR a nivel contrato"
          [,1]
[1,] -28.12459
[1] "CVaR a nivel contrato CF"
          [,1]
[1,] -28.12459
[1] "VaR a nivel contrato"
              [,1]
[1,] -23.858347703
[2,]  -0.008996254
[3,]  -0.039089948
[1] "VaR a nivel contrato CF"
              [,1]
[1,] -23.858347703
[2,]  -0.008996254
[3,]  -0.039089971
[1] "CVaR a nivel contrato"
             [,1]
[1,] -28.12361129
[2,]  -0.01060455
[3,]  -0.04607823
[1] "CVaR a nivel contrato CF"
             [,1]
[1,] -28.12361129
[2,]  -0.01060455
[3,]  -0.04607825


####Riesgo a Nivel Total y Total Factor de Riesgo

In [ ]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_ftdc=cov(DeltaX_ftdc)*(n/(n-1))

tgammavar_ftdc=(gamma_ftdctT2%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
mudg_ftdc=ifelse((sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc))>0,0,(sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)))
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))

VaRT_ftdc_DGM=(mudg_ftdc)+(desv_ftdc)*qnorm(1-alpha)  #VaR Normal
CVaRT_ftdc_DGM=-(mudg_ftdc)-(desv_ftdc)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT%*%VarDeltaX_ftdc%*%gamma_ftdctT2%*%VarDeltaX_ftdc%*%t(g_ftdctT) #gamma tercer momento
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
VaRT_ftdc_DGM_CF=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_ftdc_DGM_CF=-(mudg_ftdc)-(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))



VaRT_fr_ftdc_DGM=matrix(0,1,3)
CVaRT_fr_ftdc_DGM=matrix(0,1,3)
VaRT_fr_ftdc_DGM_CF=matrix(0,1,3)
CVaRT_fr_ftdc_DGM_CF=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))


#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_ftdctT2_fr=gamma_ftdctT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_ftdctT_fr=g_ftdctT*WFRT[j,]
# print(WFRM)
# print(g_ftdctT_fr)
# print(gamma_ftdctT2_fr)
# print(WFRT)
tgammavar_ftdc=(gamma_ftdctT2_fr%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
#mudg_ftdc=sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)
mudg_ftdc=0
desv_ftdc=sqrt(sum(diag(tgammavar2_ftdc))/2+g_ftdctT_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr))
VaRT_fr_ftdc_DGM[j]=(mudg_ftdc)+(desv_ftdc)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_ftdc_DGM[j]=-(mudg_ftdc)-(desv_ftdc)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT_fr%*%VarDeltaX_ftdc%*%gamma_ftdctT2_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr) #gamma tercer momento
 #print(asimP_ftdc)
asimP_ftdc=asimP_ftdc/(desv_ftdc)^3  #coef de asimetría
 
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
#print(VaRstd_ftdc)
VaRT_fr_ftdc_DGM_CF[j]=(mudg_ftdc)+(desv_ftdc)*VaRstd_ftdc #VaR cf
CVaRT_fr_ftdc_DGM_CF[j]=-(mudg_ftdc)-(desv_ftdc)*dnorm(VaRstd_ftdc)/(pnorm(VaRstd_ftdc))


}
print("VaR Total Delta Gamma Normal")
print(VaRT_ftdc_DGM)
print("CVaR Total Delta Gamma Normal")
print(CVaRT_ftdc_DGM)
print("VaR Total Delta Gamma Normal CF")
print(VaRT_ftdc_DGM_CF)
print("CVaR Total Delta Gamma Normal CF")
print(CVaRT_ftdc_DGM_CF)

print("VaR Total Delta Gamma Normal por factor de riesgo")
print(VaRT_fr_ftdc_DGM)
print("CVaR Total Delta Gamma Normal por factor de riesgo")
print(CVaRT_fr_ftdc_DGM)
print("VaR Total Delta Gamma Normal CF por factor de riesgo")
print(VaRT_fr_ftdc_DGM_CF)
print("CVaR Total Delta Gamma Normal CF por factor de riesgo")
print(CVaRT_fr_ftdc_DGM_CF)

[1] "VaR Total Delta Gamma Normal"
          [,1]
[1,] -23.85918
[1] "CVaR Total Delta Gamma Normal"
          [,1]
[1,] -28.12459
[1] "VaR Total Delta Gamma Normal CF"
          [,1]
[1,] -23.85918
[1] "CVaR Total Delta Gamma Normal CF"
          [,1]
[1,] -28.12459
[1] "VaR Total Delta Gamma Normal por factor de riesgo"
          [,1]         [,2]        [,3]
[1,] -23.85835 -0.008996254 -0.03908995
[1] "CVaR Total Delta Gamma Normal por factor de riesgo"
          [,1]        [,2]        [,3]
[1,] -28.12361 -0.01060455 -0.04607823
[1] "VaR Total Delta Gamma Normal CF por factor de riesgo"
          [,1]         [,2]        [,3]
[1,] -23.85835 -0.008996254 -0.03908997
[1] "CVaR Total Delta Gamma Normal CF por factor de riesgo"
          [,1]        [,2]        [,3]
[1,] -28.12361 -0.01060455 -0.04607825


###Forwards IPC

####Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [ ]:
%%R

DeltaX_find=DeltaX_port[,(sum(n_if[1:5],1)):(sum(n_if[1:6]))] 
m=ncol(plazos_fwd_ind)
m_find=ncol(plazos_fwd_ind)
VaRmargc_find=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_find=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_find=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_find=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_find=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_find=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_find=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_find=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_find=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_find=matrix(0,3,m) #CVar por contrato estandarizado
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_find=t(g_find[seq(i,m*3,by=m)]) 
  tdelta_find=t(g_find[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]   #delta tilde
  gamma_find=mgamma_futTC(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
  tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
  Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_find=(tgamma_find%*%Varmargcaux_find)
  tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
  mudg_find=ifelse((sum(diag(tgammavar_find))/2+mu_find[i])>0,0,(sum(diag(tgammavar_find))/2+mu_find[i]))
  #mudg_find=0
  desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))

  VaRmargc_find[i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_find[i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
  asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
  asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
  VaRmargcstd_find[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
  VaRmargcCF_find[i]=mudg_find+desv_find*VaRmargcstd_find[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_find[i]=-mudg_find-desv_find*(dnorm(VaRmargcstd_find[i])/(pnorm(VaRmargcstd_find[i])))

#Por factor de riesgo y por contrato

  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_find=t(g_find[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE
    tdelta_find=t(g_find[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
  	gamma_find=mgamma_futTC(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
    tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
    Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_find=(tgamma_find%*%Varmargcaux_find)
    tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
    #mudg_find=sum(diag(tgammavar_find))/2+mu_find[i]
   mudg_find=0
    desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))

    VaRmargc_fr_find[j,i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_find[j,i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
    asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
    asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
    VaRmargcstd_fr_find[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
    VaRmargcCF_fr_find[j,i]=mudg_find+desv_find*VaRmargcstd_fr_find[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_find[j,i]=-mudg_find-desv_find*(dnorm(VaRmargcstd_fr_find[j,i])/(pnorm(VaRmargcstd_fr_find[j,i])))
  }



}


print("Valor a nivel contrato")
print(V0_fwind)
print("VaR a nivel contrato")
print(VaRmargc_find)
print("VaR a nivel contrato CF")
print(VaRmargcCF_find)
print("CVaR a nivel contrato")
print(CVaRmargc_find)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_find)


print("VaR a nivel contrato")
print(VaRmargc_fr_find)
print("VaR a nivel contrato CF")
print(VaRmargcCF_fr_find)
print("CVaR a nivel contrato")
print(CVaRmargc_fr_find)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_fr_find)

[1] "Valor a nivel contrato"
         [,1]
[1,] 342308.9
[1] "VaR a nivel contrato"
          [,1]
[1,] -51116.04
[1] "VaR a nivel contrato CF"
          [,1]
[1,] -51116.04
[1] "CVaR a nivel contrato"
          [,1]
[1,] -60110.45
[1] "CVaR a nivel contrato CF"
          [,1]
[1,] -60110.45
[1] "VaR a nivel contrato"
              [,1]
[1,] -5.101370e+04
[2,] -9.430151e-03
[3,] -4.897291e+02
[1] "VaR a nivel contrato CF"
              [,1]
[1,] -5.101370e+04
[2,] -2.464920e-03
[3,] -4.897187e+02
[1] "CVaR a nivel contrato"
              [,1]
[1,] -6.013364e+04
[2,] -1.111602e-02
[3,] -5.772802e+02
[1] "CVaR a nivel contrato CF"
              [,1]
[1,] -6.013364e+04
[2,] -5.363649e-03
[3,] -5.772709e+02


####Riesgo a nivel total y total factor de riesgo

In [ ]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_find=cov(DeltaX_find)*(n/(n-1))

tgammavar_find=(gamma_findtT2%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#media
mudg_find=ifelse((sum(diag(tgammavar_find))/2+sum(mu_find))>0,0,(sum(diag(tgammavar_find))/2+mu_find[i]))
#desviación estándar
desv_find=sqrt(sum(diag(tgammavar2_find))/2+g_findtT%*%VarDeltaX_find%*%t(g_findtT))

VaRT_find_DGM=(mudg_find)+(desv_find)*qnorm(1-alpha)  #VaR Normal
CVaRT_find_DGM=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT%*%VarDeltaX_find%*%gamma_findtT2%*%VarDeltaX_find%*%t(g_findtT) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado
VaRT_find_DGM_CF=(mudg_find)+(desv_find)*VaRstd_find #VaR cf
CVaRT_find_DGM_CF=-(mudg_find)-(desv_find)*dnorm(VaRstd_find)/(pnorm(VaRstd_find))



VaRT_fr_find_DGM=matrix(0,1,3)
CVaRT_fr_find_DGM=matrix(0,1,3)
VaRT_fr_find_DGM_CF=matrix(0,1,3)
CVaRT_fr_find_DGM_CF=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_findtT2_fr=gamma_findtT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_findtT_fr=g_findtT*WFRT[j,]
tgammavar_find=(gamma_findtT2_fr%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#mudg_find=sum(diag(tgammavar_find))/2+sum(mu_find)
mudg_find=0
desv_find=sqrt(sum(diag(tgammavar2_find))/2+g_findtT_fr%*%VarDeltaX_find%*%t(g_findtT_fr))
VaRT_fr_find_DGM[j]=(mudg_find)+(desv_find)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_find_DGM[j]=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT_fr%*%VarDeltaX_find%*%gamma_findtT2_fr%*%VarDeltaX_find%*%t(g_findtT_fr) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
 
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado
VaRT_fr_find_DGM_CF[j]=(mudg_find)+(desv_find)*VaRstd_find #VaR cf
CVaRT_fr_find_DGM_CF[j]=-(mudg_find)-(desv_find)*dnorm(VaRstd_find)/(pnorm(VaRstd_find))
    
}
print("VaR Total Delta Gamma Normal")
print(VaRT_find_DGM)
print("CVaR Total Delta Gamma Normal")
print(CVaRT_find_DGM)
print("VaR Total Delta Gamma Normal CF")
print(VaRT_find_DGM_CF)
print("CVaR Total Delta Gamma Normal CF")
print(CVaRT_find_DGM_CF)

print("VaR Total Delta Gamma Normal por factor de riesgo")
print(VaRT_fr_find_DGM)
print("CVaR Total Delta Gamma Normal por factor de riesgo")
print(CVaRT_fr_find_DGM)
print("VaR Total Delta Gamma Normal CF por factor de riesgo")
print(VaRT_fr_find_DGM_CF)
print("CVaR Total Delta Gamma Normal CF por factor de riesgo")
print(CVaRT_fr_find_DGM_CF)


[1] "VaR Total Delta Gamma Normal"
          [,1]
[1,] -51116.04
[1] "CVaR Total Delta Gamma Normal"
          [,1]
[1,] -60110.45
[1] "VaR Total Delta Gamma Normal CF"
          [,1]
[1,] -51116.04
[1] "CVaR Total Delta Gamma Normal CF"
          [,1]
[1,] -60110.45
[1] "VaR Total Delta Gamma Normal por factor de riesgo"
         [,1] [,2]      [,3]
[1,] -51013.7    0 -489.7291
[1] "CVaR Total Delta Gamma Normal por factor de riesgo"
          [,1] [,2]      [,3]
[1,] -60133.64    0 -577.2802
[1] "VaR Total Delta Gamma Normal CF por factor de riesgo"
         [,1] [,2]      [,3]
[1,] -51013.7  NaN -489.7187
[1] "CVaR Total Delta Gamma Normal CF por factor de riesgo"
          [,1] [,2]      [,3]
[1,] -60133.64  NaN -577.2709


####Total futuros

In [ ]:
%%R

#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del PORTAFOLIO
DeltaX_fwdT = cbind(DeltaX_find,DeltaX_ftdc)
VarDeltaX_fwdT=as.matrix(cov(DeltaX_fwdT)*(n-1)/(n-2))


mu_fwdT=as.matrix(cbind(mu_ftdc,mu_find)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_fwdT=as.matrix(cbind(g_ftdctT,g_findtT)) #derivada con respecto las X's

#validar que ncol(X_port)=ncol(gt_port)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_fwdT=matrix(0,6,6)

gammat_fwdT[1:3,1:3]=gamma_ftdctT2
gammat_fwdT[4:6,4:6]=gamma_findtT2




#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_fwdT=(gammat_fwdT%*%VarDeltaX_fwdT)
tgammavar2_fwdT=(tgammavar_fwdT)%*%(tgammavar_fwdT) 
mu_fwdTT=ifelse((sum(diag(tgammavar_fwdT))/2+sum(mu_fwdT))>0,0,(sum(diag(tgammavar_fwdT))/2+sum(mu_fwdT)))
desv_fwdTT=(sqrt(sum(diag(tgammavar2_fwdT))/2+gt_fwdT%*%VarDeltaX_fwdT%*%t(gt_fwdT)))
VaRT_fwdT_DGM=mu_fwdTT+desv_fwdTT*qnorm(1-alpha) #VaR Normal
CVaRT_fwdT_DGM=mu_fwdTT-desv_fwdTT*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_fwdT=tgammavar2_fwdT%*%(tgammavar_fwdT)
asimP_fwdT=sum(diag(tgammavar3_fwdT))+3*gt_fwdT%*%VarDeltaX_fwdT%*%gammat_fwdT%*%VarDeltaX_fwdT%*%t(gt_fwdT) #gamma tercer momento
asimP_fwdT=asimP_fwdT/(sqrt(sum(diag(tgammavar2_fwdT))/2+gt_fwdT%*%VarDeltaX_fwdT%*%t(gt_fwdT)))^3  #coef de asimetría
VaRstd_fwdT=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_fwdT  #VaR estandarizado
VaRT_fwdT_DGM_CF=mu_fwdTT+desv_fwdTT*VaRstd_fwdT #VaR cf
CVaRT_fwdT_DGM_CF=mu_fwdTT-desv_fwdTT*dnorm(VaRstd_fwdT)/(pnorm(VaRstd_fwdT))


print("VaR forwards Total Delta Gamma Normal")
print(VaRT_fwdT_DGM)
print("CVaR forwards Total Delta Gamma Normal")
print(CVaRT_fwdT_DGM)
print("VaR forwards Total Delta Gamma CF")
print(VaRT_fwdT_DGM_CF)
print("CVaR forwards Total Delta Gamma CF")
print(CVaRT_fwdT_DGM_CF)

[1] "VaR forwards Total Delta Gamma Normal"
          [,1]
[1,] -33442.04
[1] "CVaR forwards Total Delta Gamma Normal"
          [,1]
[1,] -39408.82
[1] "VaR forwards Total Delta Gamma CF"
          [,1]
[1,] -33442.04
[1] "CVaR forwards Total Delta Gamma CF"
          [,1]
[1,] -39408.82


### Swaps

#### Contrato y contrato factor de riesgo

In [ ]:
%%R

DeltaX_sw=DeltaX_port[,(sum(n_if[1:6],1)):(sum(n_if[1:7]))] 
m=ncol(plazos_sw)
m_sw=ncol(plazos_sw)
VaRmargc_sw=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_sw=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_sw=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_sw=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcCF_sw=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#tasa cupón y tasa de valor presente
VaRmargc_fr_sw=matrix(0,2,m) #Var por contrato Delta Normal
VaRmargcstd_fr_sw=matrix(0,2,m) #Var por contrato estandarizado 
VaRmargcCF_fr_sw=matrix(0,2,m) #Var por contrato Cornish-Fisher
CVaRmargc_fr_sw=matrix(0,2,m) #CVar por contrato estandarizado
CVaRmargcCF_fr_sw=matrix(0,2,m) #CVar por contrato estandarizado

WMC=matrix(0,m,sum(N)) #variable auxiliar para construir los dummys de flujos por contrato
Nsw=c(0,cumsum(N)) #variable auxiliar para construir los dummys de flujos por contrato


for (i in 1:(m))
{
  WMC[i,(Nsw[i]+1):Nsw[i+1]]=seq(1,1) #marginalización exclusiva por los flujos del swap i
  dummy_sw=c(WMC[i,],WMC[i,]) #marginalización exclusiva por los flujos del swap i (para las dos curvas)
  tdelta_sw=gt_sw*dummy_sw   #delta tilde
  tgamma_sw=dummy_sw%*%t(dummy_sw)*tgamma_sw #gamma tilde
  Varmargcaux_sw=cov(DeltaX_sw)*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_sw=(tgamma_sw%*%Varmargcaux_sw)
  tgammavar2_sw=(tgamma_sw%*%Varmargcaux_sw)%*%(tgamma_sw%*%Varmargcaux_sw)  
  mudg_sw=ifelse((sum(diag(tgammavar_sw))/2+sum(mu_sw*WMC[i,]))>0,0,(sum(diag(tgammavar_sw))/2+sum(mu_sw*WMC[i,])))
  #mudg_sw=0
  desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+tdelta_sw%*%Varmargcaux_sw%*%t(tdelta_sw))

  VaRmargc_sw[i]=mudg_sw+desv_sw*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_sw[i]=-mudg_sw-desv_sw*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_sw=tgammavar2_sw%*%(tgamma_sw%*%Varmargcaux_sw)
  asimPmargc_sw=sum(diag(tgammavar3_sw))+3*tdelta_sw%*%Varmargcaux_sw%*%tgamma_sw%*%Varmargcaux_sw%*%t(tdelta_sw) #tercer momento
  asimPmargc_sw=asimPmargc_sw/(desv_sw)^3  #coef de asimetría
  VaRmargcstd_sw[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_sw  #VaR estandarizado
  VaRmargcCF_sw[i]=mudg_sw+desv_sw*VaRmargcstd_sw[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_sw[i]=-mudg_sw-desv_sw*(dnorm(VaRmargcstd_sw[i])/(pnorm(VaRmargcstd_sw[i])))

#Por factor de riesgo y por contrato

  for (j in 1:2)
  {
    WFRMdum=matrix(0,2,sum(N)*2)  #PASO CLAVE
    WFRMdum[1,1:(sum(N))]=1  #PASO CLAVE
    WFRMdum[2,(sum(N)+1):(sum(N)*2)]=1  #PASO CLAVE
    WFRM=WFRMdum
    tdelta_fr_sw=tdelta_sw*WFRM[j,]   #delta tilde #PASO CLAVE
    tgamma_fr_sw=tgamma_sw*WFRM[j,]%*%t(WFRM[j,]) #gamma tilde
    Varmargcaux_fr_sw=Varmargcaux_sw*WFRM[j,]%*%t(WFRM[j,]) #matriz varcov factores de riesgo de cada contrato
    tgammavar_sw=(tgamma_fr_sw%*%Varmargcaux_fr_sw)
    tgammavar2_sw=(tgamma_fr_sw%*%Varmargcaux_fr_sw)%*%(tgamma_fr_sw%*%Varmargcaux_fr_sw)  
    #mudg_sw=sum(diag(tgammavar_sw))/2+mu_sw[i]
    mudg_sw=0
    desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+tdelta_fr_sw%*%Varmargcaux_fr_sw%*%t(tdelta_sw))

    VaRmargc_fr_sw[j,i]=mudg_sw+desv_sw*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_sw[j,i]=-mudg_sw-desv_sw*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_sw=tgammavar2_sw%*%(tgamma_fr_sw%*%Varmargcaux_fr_sw)
    asimPmargc_sw=sum(diag(tgammavar3_sw))+3*tdelta_fr_sw%*%Varmargcaux_fr_sw%*%tgamma_fr_sw%*%Varmargcaux_fr_sw%*%t(tdelta_fr_sw) #tercer momento
    asimPmargc_sw=asimPmargc_sw/(desv_sw)^3  #coef de asimetría
    VaRmargcstd_fr_sw[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_sw  #VaR estandarizado
    VaRmargcCF_fr_sw[j,i]=mudg_sw+desv_sw*VaRmargcstd_fr_sw[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_sw[j,i]=-mudg_sw-desv_sw*(dnorm(VaRmargcstd_fr_sw[j,i])/(pnorm(VaRmargcstd_fr_sw[j,i])))
  }



}


print("Valor a nivel contrato")
print(V0_sw)
print("VaR a nivel contrato")
print(VaRmargc_sw)
print("VaR a nivel contrato CF")
print(VaRmargcCF_sw)
print("CVaR a nivel contrato")
print(CVaRmargc_sw)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_sw)


print("VaR a nivel contrato")
print(VaRmargc_fr_sw)
print("VaR a nivel contrato CF")
print(VaRmargcCF_fr_sw)
print("CVaR a nivel contrato")
print(CVaRmargc_fr_sw)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_fr_sw)

[1] "Valor a nivel contrato"
         [,1]     [,2]
[1,] -20224.6 80734.57
[1] "VaR a nivel contrato"
          [,1]      [,2]
[1,] -41952.79 -14800.33
[1] "VaR a nivel contrato CF"
          [,1]      [,2]
[1,] -41987.28 -14800.33
[1] "CVaR a nivel contrato"
          [,1]     [,2]
[1,] -48872.72 -17192.6
[1] "CVaR a nivel contrato CF"
          [,1]     [,2]
[1,] -48903.38 -17192.6
[1] "VaR a nivel contrato"
             [,1]         [,2]
[1,] -41620.24402 -14727.24312
[2,]    -90.32768    -50.11239
[1] "VaR a nivel contrato CF"
            [,1]         [,2]
[1,] -41620.2440 -14727.24312
[2,]    -90.5101    -50.11239
[1] "CVaR a nivel contrato"
           [,1]        [,2]
[1,] -49060.881 -17360.0983
[2,]   -106.476    -59.0712
[1] "CVaR a nivel contrato CF"
            [,1]        [,2]
[1,] -49060.8813 -17360.0983
[2,]   -106.6381    -59.0712


#### Total y total factor de riesgo

In [ ]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_sw=cov(DeltaX_sw)*(n/(n-1))

tgammavar_sw=(tgamma_sw%*%VarDeltaX_sw)
tgammavar2_sw=(tgammavar_sw)%*%(tgammavar_sw) 
#media
mudg_sw=ifelse((sum(diag(tgammavar_sw))/2+sum(mu_sw))>0,0,(sum(diag(tgammavar_sw))/2+sum(mu_sw)))
#desviación estándar
desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+gt_sw%*%VarDeltaX_sw%*%t(gt_sw))

VaRT_sw_DGM=(mudg_sw)+(desv_sw)*qnorm(1-alpha)  #VaR Normal
CVaRT_sw_DGM=-(mudg_sw)-(desv_sw)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_sw=tgammavar2_sw%*%(tgammavar_sw)
asimP_sw=sum(diag(tgammavar3_sw))+3*gt_sw%*%VarDeltaX_sw%*%tgamma_sw%*%VarDeltaX_sw%*%t(gt_sw) #gamma tercer momento
asimP_sw=asimP_sw/(desv_sw)^3  #coef de asimetría
VaRstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_sw  #VaR estandarizado
VaRT_sw_DGM_CF=(mudg_sw)+(desv_sw)*VaRstd_sw #VaR cf
CVaRT_sw_DGM_CF=-(mudg_sw)-(desv_sw)*dnorm(VaRstd_sw)/(pnorm(VaRstd_sw))


print("Valor en riesgo a nivel Total DGM")
print(VaRT_sw_DGM)
print("Valor en riesgo Condicional a nivel Total DGM")
print(CVaRT_sw_DGM)
print("Valor en riesgo a nivel Total CF")
print(VaRT_sw_DGM_CF)
print("Valor en riesgo Condicional a nivel Total CF")
print(CVaRT_sw_DGM_CF)

VaRT_fr_sw_DGM=matrix(0,1,2)
CVaRT_fr_sw_DGM=matrix(0,1,2)
VaRT_fr_sw_DGM_CF=matrix(0,1,2)
CVaRT_fr_sw_DGM_CF=matrix(0,1,2)



#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:2))
{

    WFRMdum=matrix(0,2,sum(N)*2)  #PASO CLAVE
    WFRMdum[1,1:(sum(N))]=1  #PASO CLAVE
    WFRMdum[2,(sum(N)+1):(sum(N)*2)]=1  #PASO CLAVE
    WFRM=WFRMdum    
gamma_swtT2_fr=tgamma_sw*WFRM[j,]%*%t(WFRM[j,]) #evaluación de gamma multiplicado por x^{2}
g_swtT_fr=gt_sw*WFRM[j,]
tgammavar_sw=(gamma_swtT2_fr%*%VarDeltaX_sw)
tgammavar2_sw=(tgammavar_sw)%*%(tgammavar_sw) 
#mudg_sw=sum(diag(tgammavar_sw))/2+sum(mu_sw)
mudg_sw=0
desv_sw=sqrt(sum(diag(tgammavar2_sw))/2+g_swtT_fr%*%VarDeltaX_sw%*%t(g_swtT_fr))
VaRT_fr_sw_DGM[j]=(mudg_sw)+(desv_sw)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_sw_DGM[j]=-(mudg_sw)-(desv_sw)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_sw=tgammavar2_sw%*%(tgammavar_sw)
asimP_sw=sum(diag(tgammavar3_sw))+3*g_swtT_fr%*%VarDeltaX_sw%*%gamma_swtT2_fr%*%VarDeltaX_sw%*%t(g_swtT_fr) #gamma tercer momento
asimP_sw=asimP_sw/(desv_sw)^3  #coef de asimetría
 
VaRstd_sw=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_sw  #VaR estandarizado
VaRT_fr_sw_DGM_CF[j]=(mudg_sw)+(desv_sw)*VaRstd_sw #VaR cf
CVaRT_fr_sw_DGM_CF[j]=-(mudg_sw)-(desv_sw)*dnorm(VaRstd_sw)/(pnorm(VaRstd_sw))
    
}

print("Valor en riesgo a nivel factor riesgo Delta Gamma Normal")
print(VaRT_fr_sw_DGM)
print("Valor en riesgo condicional a nivel factor riesgo Delta Gamma Normal")
print(CVaRT_fr_sw_DGM)
print("Valor en riesgo a nivel factor riesgo Delta Gamma Normal CF")
print(VaRT_fr_sw_DGM_CF)
print("Valor en riesgo condicional a nivel factor riesgo Delta Gamma Normal CF")
print(CVaRT_fr_sw_DGM_CF)


[1] "Valor en riesgo a nivel Total DGM"
          [,1]
[1,] -55864.55
[1] "Valor en riesgo Condicional a nivel Total DGM"
          [,1]
[1,] -65826.34
[1] "Valor en riesgo a nivel Total CF"
          [,1]
[1,] -55864.55
[1] "Valor en riesgo Condicional a nivel Total CF"
          [,1]
[1,] -65826.34
[1] "Valor en riesgo a nivel factor riesgo Delta Gamma Normal"
          [,1]     [,2]
[1,] -55827.97 -45.7393
[1] "Valor en riesgo condicional a nivel factor riesgo Delta Gamma Normal"
          [,1]      [,2]
[1,] -65808.59 -53.91632
[1] "Valor en riesgo a nivel factor riesgo Delta Gamma Normal CF"
          [,1]     [,2]
[1,] -55827.97 -45.7393
[1] "Valor en riesgo condicional a nivel factor riesgo Delta Gamma Normal CF"
          [,1]      [,2]
[1,] -65808.59 -53.91632


###Opciones

####Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [ ]:
%%R
#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_oir=DeltaX_port[,(sum(n_if[1:7],1)):(sum(n_if[1:8]))] 
m_oir=ncol(plazos_oir)
m=ncol(plazos_oir)

VaRmargc_oir=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcDN_oir=matrix(0,1,m) #Var por contrato Delta Normal 
VaRmargcstd_oir=matrix(0,1,m) #Var por contrato estandarizado 
VaRmargcCF_oir=matrix(0,1,m) #Var por contrato Cornish-Fisher
CVaRmargc_oir=matrix(0,1,m) #CVar por contrato estandarizado
CVaRmargcDN_oir=matrix(0,1,m) #Var por contrato Delta Normal 
CVaRmargcCF_oir=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargcDN_fr_oir=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargc_fr_oir=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcDN_fr_oir=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_oir=matrix(0,3,m) #Var por contrato estandarizado 
VaRmargcCF_fr_oir=matrix(0,3,m) #Var por contrato Cornish-Fisher
CVaRmargcDN_fr_oir=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargc_fr_oir=matrix(0,3,m) #CVar por contrato estandarizado
CVaRmargcDN_fr_oir=matrix(0,3,m) #Var por contrato Delta Normal
CVaRmargcCF_fr_oir=matrix(0,3,m) #CVar por contrato estandarizado
WFR=diagv(c(1,1,1))
WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_oir=t(delta_oir_T[seq(i,m*3,by=m)]) 
  tdelta_oir=t(delta_oir_T[seq(i,m*3,by=m)])*xp_oir[seq(i,m*3,by=m)]   #delta tilde
  gamma_oir=mgammaopc(x01[i],x02[i],K_oir[i],x03[i],plazos_oir[i],cp_oir[i],cs_oir,pr_oir,dct_oir,contratos_oir[i],nominal_oir)  #paso clave
  tgamma_oir=xp_oir[seq(i,m*3,by=m)]%*%t(xp_oir[seq(i,m*3,by=m)])*gamma_oir #gamma tilde
  Varmargcaux_oir=cov(DeltaX_oir[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_oir=(tgamma_oir%*%Varmargcaux_oir)
  tgammavar2_oir=(tgamma_oir%*%Varmargcaux_oir)%*%(tgamma_oir%*%Varmargcaux_oir)  
  mudg_oir=ifelse(sum(diag(tgammavar_oir))/2+mu_oir[i]>0,0,sum(diag(tgammavar_oir))/2+mu_oir[i])
  desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

  mudn_oir=ifelse(mu_oir[i]>0,0,mu_oir[i])
  desvdn_oir=sqrt(tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

  VaRmargcDN_oir[i]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
  CVaRmargcDN_oir[i]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

  VaRmargc_oir[i]=mudg_oir+desv_oir*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_oir[i]=mudg_oir-desv_oir*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_oir=tgammavar2_oir%*%(tgamma_oir%*%Varmargcaux_oir)
  asimPmargc_oir=sum(diag(tgammavar3_oir))+3*tdelta_oir%*%Varmargcaux_oir%*%tgamma_oir%*%Varmargcaux_oir%*%t(tdelta_oir) #tercer momento
  asimPmargc_oir=asimPmargc_oir/(desv_oir)^3  #coef de asimetría
  VaRmargcstd_oir[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_oir  #VaR estandarizado
  VaRmargcCF_oir[i]=mudg_oir+desv_oir*VaRmargcstd_oir[i] 	    #VaR Cornish-Fisher
  CVaRmargcCF_oir[i]=mudg_oir-desv_oir*(dnorm(VaRmargcstd_oir[i])/(pnorm(VaRmargcstd_oir[i])))

#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_oir=t(delta_oir_T[seq(i,m*3,by=m)])*WFR[j,]
    tdelta_oir=t(delta_oir_T[seq(i,m*3,by=m)])*xp_oir[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde
    gamma_oir=mgammaopc(x01[i],x02[i],K_oir[i],x03[i],plazos_oir[i],cp_oir[i],cs_oir,pr_oir,dct_oir,contratos_oir[i],nominal_oir)*WFRM #paso clave
   #print(WFRM)
    tgamma_oir=xp_oir[seq(i,m*3,by=m)]%*%t(xp_oir[seq(i,m*3,by=m)])*gamma_oir #gamma tilde
    Varmargcaux_oir=cov(DeltaX_oir[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_oir=(tgamma_oir%*%Varmargcaux_oir)
    tgammavar2_oir=(tgamma_oir%*%Varmargcaux_oir)%*%(tgamma_oir%*%Varmargcaux_oir)  
    #mudg_oir=sum(diag(tgammavar_oir))/2+mu_oir[i]
    mudg_oir=0
    desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

    mudn_oir=0
    desvdn_oir=sqrt(tdelta_oir%*%Varmargcaux_oir%*%t(tdelta_oir))

    VaRmargcDN_fr_oir[j,i]=mudn_oir+desvdn_oir*qnorm(1-alpha) #VaR Normal por contrato Delta Normal
    CVaRmargcDN_fr_oir[j,i]=mudn_oir-desvdn_oir*(dnorm(qnorm(1-alpha))/(1-alpha))  #CVaR Normal por contrato Delta Normal

    VaRmargc_fr_oir[j,i]=mudg_oir+desv_oir*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_oir[j,i]=mudg_oir-desv_oir*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_oir=tgammavar2_oir%*%(tgamma_oir%*%Varmargcaux_oir)
    asimPmargc_oir=sum(diag(tgammavar3_oir))+3*tdelta_oir%*%Varmargcaux_oir%*%tgamma_oir%*%Varmargcaux_oir%*%t(tdelta_oir) #tercer momento
    asimPmargc_oir=asimPmargc_oir/(desv_oir)^3  #coef de asimetría
    VaRmargcstd_fr_oir[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_oir  #VaR estandarizado
    VaRmargcCF_fr_oir[j,i]=mudg_oir+desv_oir*VaRmargcstd_fr_oir[j,i] 	    #VaR Cornish-Fisher
    CVaRmargcCF_fr_oir[j,i]=mudg_oir-desv_oir*(dnorm(VaRmargcstd_fr_oir[j,i])/(pnorm(VaRmargcstd_fr_oir[j,i])))
  }



}

print("VaR a nivel contrato Delta normal")
print(VaRmargcDN_oir)
print("CVaR a nivel contrato Delta normal")
print(CVaRmargcDN_oir)
print("VaR a nivel contrato Delta Gamma Normal")
print(VaRmargc_oir)
print("CVaR a nivel contrato Delta Gamma Normal")
print(CVaRmargc_oir)
print("VaR a nivel contrato CF")
print(VaRmargcCF_oir)
print("CVaR a nivel contrato CF")
print(CVaRmargcCF_oir)

print("VaR a nivel contrato factor de riesgo delta normal")
print(VaRmargcDN_fr_oir)
print("CVaR a nivel contrato factor de riesgo delta normal")
print(CVaRmargcDN_fr_oir)
print("VaR a nivel contrato factor de riesgo Delta Gamma Normal")
print(VaRmargc_fr_oir)
print("CVaR a nivel contrato factor de riesgo Delta Gamma Normal")
print(CVaRmargc_fr_oir)
print("VaR a nivel contrato factor de riesgo CF")
print(VaRmargcCF_fr_oir)
print("CVaR a nivel contrato factor de riesgo CF")
print(CVaRmargcCF_fr_oir)

[1] "VaR a nivel contrato Delta normal"
            [,1]        [,2]
[1,] -0.08757515 -0.04730174
[1] "CVaR a nivel contrato Delta normal"
           [,1]        [,2]
[1,] -0.1032314 -0.05575808
[1] "VaR a nivel contrato Delta Gamma Normal"
            [,1]        [,2]
[1,] -0.08757567 -0.04730179
[1] "CVaR a nivel contrato Delta Gamma Normal"
          [,1]        [,2]
[1,] -0.103232 -0.05575814
[1] "VaR a nivel contrato CF"
            [,1]        [,2]
[1,] -0.08758235 -0.04733113
[1] "CVaR a nivel contrato CF"
           [,1]        [,2]
[1,] -0.1032379 -0.05578423
[1] "VaR a nivel contrato factor de riesgo delta normal"
            [,1]          [,2]
[1,] -0.01154790 -0.0009612335
[2,] -0.06765790 -0.0365505436
[3,] -0.05238478 -0.0327790004
[1] "CVaR a nivel contrato factor de riesgo delta normal"
            [,1]         [,2]
[1,] -0.01361237 -0.001133077
[2,] -0.07975340 -0.043084848
[3,] -0.06174984 -0.038639049
[1] "VaR a nivel contrato factor de riesgo Delta Gamma Normal"
   

####Riesgo a nivel total y total factor de riesgo

In [ ]:
%%R
#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_oir=cov(DeltaX_oir)*(n/(n-1))

tgammavar_oir=(gammaT_oirT2%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
mudg_oir=ifelse(sum(diag(tgammavar_oir))/2+sum(mu_oir)>0,0,sum(diag(tgammavar_oir))/2+sum(mu_oir))
desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+deltaT_oir_T%*%VarDeltaX_oir%*%t(deltaT_oir_T))

VaRT_oir_DGM=(mudg_oir)+(desv_oir)*qnorm(1-alpha)  #VaR Normal
CVaRT_oir_DGM=(mudg_oir)-(desv_oir)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_oir=tgammavar2_oir%*%(tgammavar_oir)
asimP_oir=sum(diag(tgammavar3_oir))+3*deltaT_oir_T%*%VarDeltaX_oir%*%gammaT_oirT2%*%VarDeltaX_oir%*%t(deltaT_oir_T) #gamma tercer momento
asimP_oir=asimP_oir/(desv_oir)^3  #coef de asimetría
VaRstd_oir=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_oir  #VaR estandarizado
VaRT_oir_DGM_CF=(mudg_oir)+(desv_oir)*VaRstd_oir #VaR cf
CVaRT_oir_DGM_CF=(mudg_oir)-(desv_oir)*dnorm(VaRstd_oir)/(pnorm(VaRstd_oir))

#print(VaRT_oir_DGM)
#print(CVaRT_oir_DGM)
#print(VaRT_oir_DGM_CF)
#print(CVaRT_oir_DGM_CF)

VaRT_fr_oir_DGM=matrix(0,1,3)
CVaRT_fr_oir_DGM=matrix(0,1,3)
VaRT_fr_oir_DGM_CF=matrix(0,1,3)
CVaRT_fr_oir_DGM_CF=matrix(0,1,3)

WFRT=matrix(0,3,(m*3))
# WFRT[1,1:m]=matrix(1,1,m)
# WFRT[2,(m+1):(2*m)]=matrix(1,1,m)
# WFRT[3,(2*m+1):(3*m)]=matrix(1,1,m)
#print(WFRT)

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_oirtT2_fr=gammaT_oirT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_oirtT_fr=deltaT_oir_T*WFRT[j,]
# print(WFRM)
# print(g_oirtT_fr)
# print(gamma_oirtT2_fr)
# print(WFRT)
tgammavar_oir=(gamma_oirtT2_fr%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
#mudg_oir=sum(diag(tgammavar_oir))/2+sum(mu_oir)
mudg_oir=0
desv_oir=sqrt(sum(diag(tgammavar2_oir))/2+g_oirtT_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr))
VaRT_fr_oir_DGM[j]=(mudg_oir)+(desv_oir)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_oir_DGM[j]=(mudg_oir)-(desv_oir)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_oir=tgammavar2_oir%*%(tgammavar_oir)
asimP_oir=sum(diag(tgammavar3_oir))+3*g_oirtT_fr%*%VarDeltaX_oir%*%gamma_oirtT2_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr) #gamma tercer momento
 #print(asimP_oir)
asimP_oir=asimP_oir/(desv_oir)^3  #coef de asimetría
 
VaRstd_oir=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_oir  #VaR estandarizado
#print(VaRstd_oir)
VaRT_fr_oir_DGM_CF[j]=(mudg_oir)+(desv_oir)*VaRstd_oir #VaR cf
CVaRT_fr_oir_DGM_CF[j]=(mudg_oir)-(desv_oir)*dnorm(VaRstd_oir)/(pnorm(VaRstd_oir))


}

print("VaR total por factor de riesgo delta Gamma Normal")
print(VaRT_fr_oir_DGM)
print("CVaR total por factor de riesgo delta Gamma Normal")
print(CVaRT_fr_oir_DGM)
print("VaR total por factor de riesgo CF")
print(VaRT_fr_oir_DGM_CF)
print("CVaR total por factor de riesgo CF")
print(CVaRT_fr_oir_DGM_CF)



print("Valor en riesgo a nivel Total DGN")
print(VaRT_oir_DGM)
print("Valor en riesgo Condicional a nivel Total DGN")
print(CVaRT_oir_DGM)
print("Valor en riesgo a nivel Total CF")
print(VaRT_oir_DGM_CF)
print("Valor en riesgo Condicional a nivel Total CF")
print(CVaRT_oir_DGM_CF)

[1] "VaR total por factor de riesgo delta normal"
            [,1]        [,2]        [,3]
[1,] -0.01243336 -0.08778916 -0.07886968
[1] "CVaR total por factor de riesgo delta normal"
            [,1]       [,2]        [,3]
[1,] -0.01465613 -0.1034836 -0.09296956
[1] "VaR total por factor de riesgo CF"
            [,1]        [,2]        [,3]
[1,] -0.01238144 -0.08778654 -0.07895535
[1] "CVaR total por factor de riesgo CF"
            [,1]       [,2]        [,3]
[1,] -0.01460999 -0.1034813 -0.09304572
[1] "Valor en riesgo a nivel Total DGM"
           [,1]
[1,] -0.1182985
[1] "Valor en riesgo Condicional a nivel Total DGM"
           [,1]
[1,] -0.1394473
[1] "Valor en riesgo a nivel Total CF"
           [,1]
[1,] -0.1183589
[1] "Valor en riesgo Condicional a nivel Total CF"
           [,1]
[1,] -0.1395009


### Riesgo Integral

##### Total

In [ ]:
%%R
#Cálculo de matriz de varianza covarianza de todos los factores de riesgo del PORTAFOLIO
VarDeltaX_port=as.matrix(cov(DeltaX_port)*(n-1)/(n-2))

mu_port=as.matrix(cbind(mu_acc_div,mu_bcc,mu_bm,mu_bd,mu_ftdc,mu_find,mu_sw,mu_oir)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T)) #derivada con respecto las X's
#validar que ncol(X_port)=ncol(gt_port)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2



#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_DGM=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_DGM=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port%*%VarDeltaX_port%*%gammat_port%*%VarDeltaX_port%*%t(gt_port) #gamma tercer momento
asimP_port=asimP_port/(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_DGM_CF=mu_portT+desv_portT*VaRstd_port #VaR cf
CVaRT_port_DGM_CF=mu_portT-desv_portT*dnorm(VaRstd_port)/(pnorm(VaRstd_port))



print(VaRT_port_DGM)
print(CVaRT_port_DGM)

print(VaRT_port_DGM_CF)
print(CVaRT_port_DGM_CF)



          [,1]
[1,] -76468.43
          [,1]
[1,] -90130.61
         [,1]
[1,] -76468.4
          [,1]
[1,] -90130.59


In [ ]:
%%R
sum(n_if[1:5])

[1] 167


#### Por acciones

In [ ]:
%%R
m = ncol(plazos_fwd_ind)
WFRT_acc=matrix(0,3,(m*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT_acc[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRT_acc=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRT_acc[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)}

dummy_acc = matrix(0,1, ncol(gt_port))
dummy_acc[(m_fx+1):(m_fx+m_acc)] = 1
dummy_acc[c(168)] = 1 #Cambia dependiendo del factor de riesgo a tomar. Revisar los riesgos de las opciones IPC
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T)) #derivada con respecto las X's

gt_port = gt_port*dummy_acc
gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2

gammat_port = gammat_port * t(dummy_acc)%*%dummy_acc

tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_acc_DGM=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_acc_DGM=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port%*%VarDeltaX_port%*%gammat_port%*%VarDeltaX_port%*%t(gt_port) #gamma tercer momento
asimP_port=asimP_port/(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_acc_DGM_CF=mu_portT+desv_portT*VaRstd_port #VaR cf
CVaRT_port_acc_DGM_CF=mu_portT-desv_portT*dnorm(VaRstd_port)/(pnorm(VaRstd_port))



print(VaRT_port_acc_DGM)
print(CVaRT_port_acc_DGM)

print(VaRT_port_acc_DGM_CF)
print(CVaRT_port_acc_DGM_CF)

          [,1]
[1,] -52688.68
          [,1]
[1,] -62099.28
          [,1]
[1,] -52688.68
          [,1]
[1,] -62099.28


#### Divisas

In [ ]:
%%R
m = ncol(plazos_fwd)
WFRT_div=matrix(0,3,(m*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT_div[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRT_div=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRT_div[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)}

dummy_div = matrix(0,1, ncol(gt_port))
dummy_div[1:m_fx] = 1
dummy_div[c(165, 166, 167)] = 1 #Cambia dependiendo del factor de riesgo a tomar.
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T)) #derivada con respecto las X's

gt_port = gt_port*dummy_div
gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2

gammat_port = gammat_port * t(dummy_div)%*%dummy_div

tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_div_DGM=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_div_DGM=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port%*%VarDeltaX_port%*%gammat_port%*%VarDeltaX_port%*%t(gt_port) #gamma tercer momento
asimP_port=asimP_port/(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_div_DGM_CF=mu_portT+desv_portT*VaRstd_port #VaR cf
CVaRT_port_div_DGM_CF=mu_portT-desv_portT*dnorm(VaRstd_port)/(pnorm(VaRstd_port))



print(VaRT_port_div_DGM)
print(CVaRT_port_div_DGM)

print(VaRT_port_div_DGM_CF)
print(CVaRT_port_div_DGM_CF)

          [,1]
[1,] -449.7078
          [,1]
[1,] -521.3069
          [,1]
[1,] -449.7078
          [,1]
[1,] -521.3069


#### Tasa de interés

In [ ]:
%%R

dummy_tint = matrix(0,1, ncol(gt_port))
dummy_tint[7:sum(n_if[2:3], n_if[4])] = 1
dummy_tint[c(169, 170)] = 1
dummy_tint[sum(n_if[1:7],1):length(dummy_tint)] = 1 #Cambia dependiendo del factor de riesgo a tomar.
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T)) #derivada con respecto las X's

gt_port = gt_port*dummy_tint
gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2

gammat_port = gammat_port * t(dummy_tint)%*%dummy_tint

tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_tint_DGM=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_tint_DGM=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_port=tgammavar2_port%*%(tgammavar_port)
asimP_port=sum(diag(tgammavar3_port))+3*gt_port%*%VarDeltaX_port%*%gammat_port%*%VarDeltaX_port%*%t(gt_port) #gamma tercer momento
asimP_port=asimP_port/(sqrt(sum(diag(tgammavar2_port))/2+gt_port%*%VarDeltaX_port%*%t(gt_port)))^3  #coef de asimetría
VaRstd_port=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_port  #VaR estandarizado
VaRT_port_tint_DGM_CF=mu_portT+desv_portT*VaRstd_port #VaR cf
CVaRT_port_tint_DGM_CF=mu_portT-desv_portT*dnorm(VaRstd_port)/(pnorm(VaRstd_port))



print(VaRT_port_tint_DGM)
print(CVaRT_port_tint_DGM)

print(VaRT_port_tint_DGM_CF)
print(CVaRT_port_tint_DGM_CF)

          [,1]
[1,] -1347.566
          [,1]
[1,] -1580.049
          [,1]
[1,] -1341.849
          [,1]
[1,] -1574.969


## Delta-Normal

### Acciones y divisas

#### Por factor de riesgo

In [ ]:
%%R
VaRDNfr_acc = qnorm(1-alpha)*sqrt(g_acct*g_acct*desvest_acc*desvest_acc) 
VaRDNfr_div = qnorm(1-alpha)*sqrt(g_divt*g_divt*desvest_div*desvest_div) 

CVaRDNfr_acc = (dnorm(qnorm(1-alpha))/(1-alpha))*sqrt(g_acct*g_acct*desvest_acc*desvest_acc)*-1
CVaRDNfr_div = (dnorm(qnorm(1-alpha))/(1-alpha))*sqrt(g_divt*g_divt*desvest_div*desvest_div)*-1

print('VaR por factor de riesgo de acciones Delta Normal')
print(VaRDNfr_acc)
print('CVaR por factor de riesgo de acciones Delta Normal')
print(CVaRDNfr_acc)
print('VaR por factor de riesgo de divisas Delta Normal')
print(VaRDNfr_div)
print('CVaR por factor de riesgo de divisas Delta Normal')
print(CVaRDNfr_div)

[1] "VaR por factor de riesgo de acciones Delta Normal"
     AMXL.MX GCARSOA1.MX WALMEX.MX
1: -2864.347   -3468.902 -2978.942
[1] "CVaR por factor de riesgo de acciones Delta Normal"
     AMXL.MX GCARSOA1.MX WALMEX.MX
1: -3376.419   -4089.053 -3511.501
[1] "VaR por factor de riesgo de divisas Delta Normal"
    EURUSD.X  GBPUSD.X  USDMXN.X
1: -157.1863 -159.9017 -357.3584
[1] "CVaR por factor de riesgo de divisas Delta Normal"
    EURUSD.X GBPUSD.X  USDMXN.X
1: -185.2872 -188.488 -421.2449


#### Total

In [ ]:
%%R
VaRDNTotal_acc = sqrt(as.matrix(g_acct)%*%VarCovX_acc%*%t(as.matrix(g_acct)))*qnorm(1-alpha)
CVaRDNTotal_acc = sqrt(as.matrix(g_acct)%*%VarCovX_acc%*%t(as.matrix(g_acct)))*dnorm(qnorm(1-alpha))/(1-alpha)*-1

VaRDNTotal_div = sqrt(as.matrix(g_divt)%*%VarCovX_div%*%t(as.matrix(g_divt)))*qnorm(1-alpha)
CVaRDNTotal_div = sqrt(as.matrix(g_divt)%*%VarCovX_div%*%t(as.matrix(g_divt)))*dnorm(qnorm(1-alpha))/(1-alpha)*-1

print('VaR Total de acciones Delta Normal')
print(VaRDNTotal_acc)
print('CVaR Total de acciones Delta Normal')
print(CVaRDNTotal_acc)
print('VaR Total de divisas Delta Normal')
print(VaRDNTotal_div)
print('CVaR Total de divisas Delta Normal')
print(CVaRDNTotal_div)

[1] "VaR Total de acciones Delta Normal"
          [,1]
[1,] -4824.416
[1] "CVaR Total de acciones Delta Normal"
          [,1]
[1,] -5686.899
[1] "VaR Total de divisas Delta Normal"
          [,1]
[1,] -376.8845
[1] "CVaR Total de divisas Delta Normal"
          [,1]
[1,] -444.2619


### CETES

In [ ]:
%%R
VaRDN_bcc = (mu_bcc[1,1] + qnorm(1-alpha)*sqrt(g_bcct[1,1]^2*desvest_bcc^2))
print(VaRDN_bcc)
CVaRDN_bcc = (mu_bcc[1,1] + dnorm(qnorm(1-alpha))/(1-alpha)*sqrt(g_bcct[1,1]^2*desvest_bcc^2)*-1)
print(CVaRDN_bcc)

[1] -10.18972
[1] -11.4639


### Bondes D

#### Por contrato

In [ ]:
%%R
DeltaX_bd_tc=X_bd_tc[1:(n-1),]/X_bd_tc[2:n,]-1
DeltaXvp_bd=Xvp_bd[1:(n-1),]/Xvp_bd[2:n,]-1
DeltaXst_bdt=Xst_bd[1:(n-1),]/Xst_bd[2:n,]-1

VaRcontDN_bd=matrix(0,1,count(N_bd))
CVaRcontDN_bd=matrix(0,1,count(N_bd))

for (j in (1:count(N_bd)))
{
  if(j==1)
  {
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[1:sum(N_bd[1:j])],V0f_dvp_bd[1:sum(N_bd[1:j])],V0f_dst_bd[1:sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(DeltaX_bd_tc[,1:sum(N_bd[1:j])],DeltaXvp_bd[,1:sum(N_bd[1:j])],DeltaXst_bdt[,1:sum(N_bd[1:j])]))*(n-1)/(n-2)	
  }
  else
  {
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dvp_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dst_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*x0cont_bd))
    VarCovXcont_bd=cov(cbind(DeltaX_bd_tc[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],DeltaXvp_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],DeltaXst_bdt[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))*(n-1)/(n-2)
  }
  
  
  
  #VaR Delta Normal
  VaRcontDN_bd[j]=sum(mucont_bd)+sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))*qnorm(1-alpha)
  CVaRcontDN_bd[j]=sum(mucont_bd)+sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))*dnorm(qnorm(1-alpha))/(1-alpha)*-1
}

print("VaR por contrato Delta Normal")
print(VaRcontDN_bd[1,1])
print("CVaR por contrato Delta Normal")
print(CVaRcontDN_bd[1,1])

[1] "VaR por contrato Delta Normal"
[1] -442.7493
[1] "CVaR por contrato Delta Normal"
[1] -526.1521


#### Por contrato factor de riesgo total

In [ ]:
%%R
WFR_bd=rbind(c(X_bd_tc[1,],Xvp_bd[1,], Xst_bd[1,]*0),c( X_bd_tc[1,],Xvp_bd[1,]*0, Xst_bd[1,]), c( X_bd_tc[1,]*0,Xvp_bd[1,], Xst_bd[1,]))
VaRFrDN_bd=matrix(0,1,3)
CVaRFrDN_bd=matrix(0,1,3)

for (j in (1:3))
{ 
    g_bd=cbind(V0f_dtc_bd, V0f_dvp_bd, V0f_dst_bd)
    
    #Multiplicación de la derivada por el punto x0
    gt_bd=WFR_bd[j,]*g_bd
    VaRFrDN_bd[j]=sqrt(gt_bd%*%VarCovX_bd_ext%*%t(gt_bd))*qnorm(1-alpha)
    CVaRFrDN_bd[j]=-sqrt(gt_bd%*%VarCovX_bd_ext%*%t(gt_bd))*dnorm(qnorm(1-alpha))/(1-alpha)
}
print("VaR por contrato fr total")
print(VaRFrDN_bd)
print("CVaR por contrato fr total")
print(CVaRFrDN_bd)


[1] "VaR por contrato fr total"
          [,1]      [,2]      [,3]
[1,] -421.5042 -371.7574 -300.5735
[1] "CVaR por contrato fr total"
          [,1]      [,2]      [,3]
[1,] -496.8584 -438.2181 -354.3083


#### Contrato Factor de riesgo marginal

In [ ]:
%%R
VaRcontfrDN_bd=matrix(0,3,count(N_bd))
CVaRcontfrDN_bd=matrix(0,3,count(N_bd))
WFRcont_bd <- rbind(c(1,0,0), c(0,1,0), c(0,0,1))

for (j in (1:count(N_bd)))
{
    for(k in (1:3)){
        if(j==1)
  {
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[1:sum(N_bd[1:j])],V0f_dvp_bd[1:sum(N_bd[1:j])],V0f_dst_bd[1:sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*c(X_bd_tc[1,1:sum(N_bd[1:j])]*WFRcont_bd[k,1],Xvp_bd[1,1:sum(N_bd[1:j])]*WFRcont_bd[k,2], Xst_bd[1,1:sum(N_bd[1:j])]*WFRcont_bd[k,3])))
    VarCovXcont_bd=cov(cbind(DeltaX_bd_tc[,1:sum(N_bd[1:j])],DeltaXvp_bd[,1:sum(N_bd[1:j])],DeltaXst_bdt[,1:sum(N_bd[1:j])]))*(n-1)/(n-2)	
  }
  else
  {
    mucont_bd=sum(mu_bd[j])
    gcont_bd=as.matrix(c(V0f_dtc_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dvp_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])],V0f_dst_bd[sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))
    gtcont_bd=t(as.matrix(gcont_bd*c(X_bd_tc[1,sum(N_bd[1:j-1]+1):sum(N_bd[1:j])]*WFRcont_bd[k,1],Xvp_bd[1,sum(N_bd[1:j-1]+1):sum(N_bd[1:j])]*WFRcont_bd[k,2], Xst_bd[1,sum(N_bd[1:j-1]+1):sum(N_bd[1:j])]*WFRcont_bd[k,3])))
    VarCovXcont_bd=cov(cbind(DeltaX_bd_tc[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],DeltaXvp_bd[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])],DeltaXst_bdt[,sum(N_bd[1:j-1],1):sum(N_bd[1:j])]))*(n-1)/(n-2)
  }

  #VaR Delta Normal
  VaRcontfrDN_bd[k,j]=sum(mucont_bd)+sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))*qnorm(1-alpha)
  CVaRcontfrDN_bd[k,j]=sum(mucont_bd)-sqrt(gtcont_bd%*%VarCovXcont_bd%*%t(gtcont_bd))*dnorm(qnorm(1-alpha))/(1-alpha)
}
}

print("VaR por contrato fr Delta Normal")
print(VaRcontfrDN_bd[,1])
print("CVaR por contrato fr Delta Normal")
print(CVaRcontfrDN_bd[,1])

[1] "VaR por contrato fr Delta Normal"
[1] -341.54676 -196.99378  -56.02897
[1] "CVaR por contrato fr Delta Normal"
[1] -406.85707 -236.46171  -70.29599


#### Total

In [ ]:
%%R
#Evalución de las derivada en el punto x0

g_bd=cbind(V0f_dtc_bd, V0f_dvp_bd, V0f_dst_bd)

#Multiplicación de la derivada por el punto x0
gt_bd= c(X_bd_tc[1,],Xvp_bd[1,], Xst_bd[1,])*g_bd
VaRTotalDN_bd=sqrt(gt_bd%*%VarCovX_bd_ext%*%t(gt_bd))*qnorm(1-alpha)+sum(d_bondeD_t(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv))
CVaRTotalDN_bd = -sqrt(gt_bd%*%VarCovX_bd_ext%*%t(gt_bd))*dnorm(qnorm(1-alpha))/(1-alpha)+sum(d_bondeD_t(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[1,], Xst_bd[1,], N_bd,ddv))


print("VaR Total Delta Normal")
print(VaRTotalDN_bd)
print("CVaR Total Delta Normal")
print(CVaRTotalDN_bd)

[1] "VaR Total Delta Normal"
          [,1]
[1,] -442.7493
[1] "CVaR Total Delta Normal"
          [,1]
[1,] -526.1521


###Forwards de tipo de cambio

####Por contrato

In [ ]:
%%R



#Cálculo de matriz de pérdidas y ganancias FUTUROS TDC

#Cálculo de VaR por contrato
#creación de matrices de VaR, gamma, VaRCF por contrato
DeltaX_ftdc=DeltaX_port[,(sum(n_if[1:4],1)):(sum(n_if[1:5]))] 
m_ftdc=ncol(plazos_fwd)
m=ncol(plazos_fwd)
VaRmargc_ftdc=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_ftdc=matrix(0,1,m) #Var por contrato estandarizado 

CVaRmargc_ftdc=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_ftdc=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_ftdc=matrix(0,3,m) #Var por contrato estandarizado 

CVaRmargc_fr_ftdc=matrix(0,3,m) #CVar por contrato estandarizado

WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)]) 
  tdelta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]   #delta tilde
  gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)  #paso clave
  tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
  Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
  tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
  mudg_ftdc=ifelse((sum(diag(tgammavar_ftdc))/2+mu_ftdc[i])>0,0,(sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]))
  desv_ftdc=sqrt(tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))

  VaRmargc_ftdc[i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_ftdc[i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
  asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
  asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
  VaRmargcstd_ftdc[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
  

#Por factor de riesgo y por contrato
#spot, tasa extranjera, tasa nacional
  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE *(1,0,0), (0,1,0), (0,0,1)
    #print(WFR[j,])
    tdelta_ftdc=t(g_ftdc[seq(i,m*3,by=m)])*xp_ftdc[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
    gamma_ftdc=mgamma_futTC(plazos_fwd[i],X1_fwtdc[1,i],X2_fwtdc[1,i],X3_ftdc[1,],kst_fwd[i],contratos_fwd[i],nominal_fwd)*WFRM  #paso clave
   #print(WFRM)
    tgamma_ftdc=xp_ftdc[seq(i,m*3,by=m)]%*%t(xp_ftdc[seq(i,m*3,by=m)])*gamma_ftdc #gamma tilde
    Varmargcaux_ftdc=cov(DeltaX_ftdc[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)
    tgammavar2_ftdc=(tgamma_ftdc%*%Varmargcaux_ftdc)%*%(tgamma_ftdc%*%Varmargcaux_ftdc)  
    #mudg_ftdc=sum(diag(tgammavar_ftdc))/2+mu_ftdc[i]
    mudg_ftdc=0
    desv_ftdc=sqrt(tdelta_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc))

    VaRmargc_fr_ftdc[j,i]=mudg_ftdc+desv_ftdc*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_ftdc[j,i]=-mudg_ftdc-desv_ftdc*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_ftdc=tgammavar2_ftdc%*%(tgamma_ftdc%*%Varmargcaux_ftdc)
    asimPmargc_ftdc=sum(diag(tgammavar3_ftdc))+3*tdelta_ftdc%*%Varmargcaux_ftdc%*%tgamma_ftdc%*%Varmargcaux_ftdc%*%t(tdelta_ftdc) #tercer momento
    asimPmargc_ftdc=asimPmargc_ftdc/(desv_ftdc)^3  #coef de asimetría
    VaRmargcstd_fr_ftdc[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_ftdc  #VaR estandarizado
    
  }



}
print("Valor a nivel contrato")
print(V0_fwtdc)
print("VaR a nivel contrato")
print(VaRmargc_ftdc)

print("CVaR a nivel contrato")
print(CVaRmargc_ftdc)



print("VaR a nivel contrato")
print(VaRmargc_fr_ftdc)

print("CVaR a nivel contrato")
print(CVaRmargc_fr_ftdc)



[1] "Valor a nivel contrato"
          [,1]
[1,] -85.61414
[1] "VaR a nivel contrato"
          [,1]
[1,] -23.85918
[1] "CVaR a nivel contrato"
          [,1]
[1,] -28.12459
[1] "VaR a nivel contrato"
              [,1]
[1,] -23.858347703
[2,]  -0.008996254
[3,]  -0.039089948
[1] "CVaR a nivel contrato"
             [,1]
[1,] -28.12361129
[2,]  -0.01060455
[3,]  -0.04607823


####Total





In [ ]:

%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_ftdc=cov(DeltaX_ftdc)*(n/(n-1))

tgammavar_ftdc=(gamma_ftdctT2%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
mudg_ftdc=ifelse((sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc))>0,0,(sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)))
desv_ftdcDN=sqrt(g_ftdctT%*%VarDeltaX_ftdc%*%t(g_ftdctT))

VaRT_ftdc_DN=(mudg_ftdc)+(desv_ftdcDN)*qnorm(1-alpha)  #VaR Normal
CVaRT_ftdc_DN=-(mudg_ftdc)-(desv_ftdcDN)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT%*%VarDeltaX_ftdc%*%gamma_ftdctT2%*%VarDeltaX_ftdc%*%t(g_ftdctT) #gamma tercer momento
asimP_ftdc=asimP_ftdc/(desv_ftdcDN)^3  #coef de asimetría
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado





VaRT_fr_ftdc_DN=matrix(0,1,3)
CVaRT_fr_ftdc_DN=matrix(0,1,3)


WFRT=matrix(0,3,(m*3))
# WFRT[1,1:m]=matrix(1,1,m)
# WFRT[2,(m+1):(2*m)]=matrix(1,1,m)
# WFRT[3,(2*m+1):(3*m)]=matrix(1,1,m)


#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_ftdctT2_fr=gamma_ftdctT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_ftdctT_fr=g_ftdctT*WFRT[j,]
# print(WFRM)
# print(g_ftdctT_fr)
# print(gamma_ftdctT2_fr)
# print(WFRT)
tgammavar_ftdc=(gamma_ftdctT2_fr%*%VarDeltaX_ftdc)
tgammavar2_ftdc=(tgammavar_ftdc)%*%(tgammavar_ftdc) 
#mudg_ftdc=sum(diag(tgammavar_ftdc))/2+sum(mu_ftdc)
mudg_ftdc=0
desv_ftdcDN=sqrt(g_ftdctT_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr))
VaRT_fr_ftdc_DN[j]=(mudg_ftdc)+(desv_ftdcDN)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_ftdc_DN[j]=-(mudg_ftdc)-(desv_ftdcDN)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_ftdc=tgammavar2_ftdc%*%(tgammavar_ftdc)
asimP_ftdc=sum(diag(tgammavar3_ftdc))+3*g_ftdctT_fr%*%VarDeltaX_ftdc%*%gamma_ftdctT2_fr%*%VarDeltaX_ftdc%*%t(g_ftdctT_fr) #gamma tercer momento
 #print(asimP_ftdc)
asimP_ftdc=asimP_ftdc/(desv_ftdcDN)^3  #coef de asimetría
 
VaRstd_ftdc=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_ftdc  #VaR estandarizado
#print(VaRstd_ftdc)



}
print("VaR total Delta Normal")
print(VaRT_ftdc_DN)
print("CVaR total Delta Normal")
print(CVaRT_ftdc_DN)
print("VaR total Delta Normal por factor de riesgo")
print(VaRT_fr_ftdc_DN)
print("CVaR total Delta Normal por factor de riesgo")
print(CVaRT_fr_ftdc_DN)





# cbind(VaRTotal_fwtdc,sum(V0_fwtdc), VaRCont_fwtdc, V0_fwtdc)
# cbind(CVaRTotal_fwtdc,sum(V0_fwtdc), CVaRCont_fwtdc, V0_fwtdc)
# cbind(VaRTotal_fwtdc,VaRTotalfr1_fwtdc,VaRTotalfr2_fwtdc,VaRTotalfr3_fwtdc)
# cbind(CVaRTotal_fwtdc,CVaRTotalfr1_fwtdc,CVaRTotalfr2_fwtdc,CVaRTotalfr3_fwtdc)

[1] "VaR total Delta Normal"
          [,1]
[1,] -23.85918
[1] "CVaR total Delta Normal"
          [,1]
[1,] -28.12459
[1] "VaR total Delta Normal por factor de riesgo"
          [,1]         [,2]        [,3]
[1,] -23.85835 -0.008996254 -0.03908995
[1] "CVaR total Delta Normal por factor de riesgo"
          [,1]        [,2]        [,3]
[1,] -28.12361 -0.01060455 -0.04607823


###Forwards IPC

#####Por contrato


In [ ]:
%%R

DeltaX_find=DeltaX_port[,(sum(n_if[1:5],1)):(sum(n_if[1:6]))] 
m=ncol(plazos_fwd_ind)
m_find=ncol(plazos_fwd_ind)
VaRmargc_find=matrix(0,1,m) #Var por contrato Delta Normal
VaRmargcstd_find=matrix(0,1,m) #Var por contrato estandarizado 
CVaRmargc_find=matrix(0,1,m) #CVar por contrato estandarizado

##factores riesgo
#Spot
VaRmargc_fr_find=matrix(0,3,m) #Var por contrato Delta Normal
VaRmargcstd_fr_find=matrix(0,3,m) #Var por contrato estandarizado 

CVaRmargc_fr_find=matrix(0,3,m) #CVar por contrato estandarizado

WFR=diagv(c(1,1,1))


for (i in 1:(m))
{
  delta_find=t(g_find[seq(i,m*3,by=m)]) 
  tdelta_find=t(g_find[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]   #delta tilde
  gamma_find=mgamma_futTC(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
  tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
  Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
  tgammavar_find=(tgamma_find%*%Varmargcaux_find)
  tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
  mudg_find=ifelse((sum(diag(tgammavar_find))/2+mu_find[i])>0,0,(sum(diag(tgammavar_find))/2+mu_find[i]))
  #mudg_find=0
  desv_find=sqrt(tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))

  VaRmargc_find[i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
  CVaRmargc_find[i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
  
  tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
  asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
  asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
  VaRmargcstd_find[i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
  

#Por factor de riesgo y por contrato

  for (j in 1:3)
  {
    WFRM=matrix(0,3,3)  #PASO CLAVE
    WFRM[j,j]=1  #PASO CLAVE
    delta_find=t(g_find[seq(i,m*3,by=m)])*WFR[j,] #PASO CLAVE
    tdelta_find=t(g_find[seq(i,m*3,by=m)])*xp_find[seq(i,m*3,by=m)]*WFR[j,]   #delta tilde #PASO CLAVE
  	gamma_find=mgamma_futTC(plazos_fwd_ind[i],X1_fwind[1,i],X2_fwind[1,i],X3_find[1,],kst_fwd_ind[i],contratos_fwd_ind[i],nominal_fwd_ind)  #paso clave
    tgamma_find=xp_find[seq(i,m*3,by=m)]%*%t(xp_find[seq(i,m*3,by=m)])*gamma_find #gamma tilde
    Varmargcaux_find=cov(DeltaX_find[,seq(i,m*3,by=m)])*((n)/(n-1)) #matriz varcov factores de riesgo de cada contrato
    tgammavar_find=(tgamma_find%*%Varmargcaux_find)
    tgammavar2_find=(tgamma_find%*%Varmargcaux_find)%*%(tgamma_find%*%Varmargcaux_find)  
    #mudg_find=sum(diag(tgammavar_find))/2+mu_find[i]
   mudg_find=0
    desv_find=sqrt(sum(diag(tgammavar2_find))/2+tdelta_find%*%Varmargcaux_find%*%t(tdelta_find))

    VaRmargc_fr_find[j,i]=mudg_find+desv_find*qnorm(1-alpha) #VaR Normal por contrato
    CVaRmargc_fr_find[j,i]=-mudg_find-desv_find*(dnorm(qnorm(1-alpha))/(1-alpha))
    
    tgammavar3_find=tgammavar2_find%*%(tgamma_find%*%Varmargcaux_find)
    asimPmargc_find=sum(diag(tgammavar3_find))+3*tdelta_find%*%Varmargcaux_find%*%tgamma_find%*%Varmargcaux_find%*%t(tdelta_find) #tercer momento
    asimPmargc_find=asimPmargc_find/(desv_find)^3  #coef de asimetría
    VaRmargcstd_fr_find[j,i]=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimPmargc_find  #VaR estandarizado
    
  }



}


print("Valor a nivel contrato")
print(V0_fwind)
print("VaR a nivel contrato")
print(VaRmargc_find)

print("CVaR a nivel contrato")
print(CVaRmargc_find)

print("VaR a nivel contrato")
print(VaRmargc_fr_find)

print("CVaR a nivel contrato")
print(CVaRmargc_fr_find)


[1] "Valor a nivel contrato"
         [,1]
[1,] 342308.9
[1] "VaR a nivel contrato"
          [,1]
[1,] -51116.04
[1] "CVaR a nivel contrato"
          [,1]
[1,] -60110.45
[1] "VaR a nivel contrato"
              [,1]
[1,] -5.101370e+04
[2,] -9.430151e-03
[3,] -4.897291e+02
[1] "CVaR a nivel contrato"
              [,1]
[1,] -6.013364e+04
[2,] -1.111602e-02
[3,] -5.772802e+02


####Total


In [ ]:
%%R


#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO

VarDeltaX_find=cov(DeltaX_find)*(n/(n-1))

tgammavar_find=(gamma_findtT2%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#media
mudg_find=ifelse((sum(diag(tgammavar_find))/2+sum(mu_find))>0,0,(sum(diag(tgammavar_find))/2+mu_find[i]))
#desviación estándar
desv_find=sqrt(g_findtT%*%VarDeltaX_find%*%t(g_findtT))

VaRT_find_DN=(mudg_find)+(desv_find)*qnorm(1-alpha)  #VaR Normal
CVaRT_find_DN=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT%*%VarDeltaX_find%*%gamma_findtT2%*%VarDeltaX_find%*%t(g_findtT) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado





VaRT_fr_find_DN=matrix(0,1,3)
CVaRT_fr_find_DN=matrix(0,1,3)


WFRT=matrix(0,3,(m*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_findtT2_fr=gamma_findtT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_findtT_fr=g_findtT*WFRT[j,]
tgammavar_find=(gamma_findtT2_fr%*%VarDeltaX_find)
tgammavar2_find=(tgammavar_find)%*%(tgammavar_find) 
#mudg_find=sum(diag(tgammavar_find))/2+sum(mu_find)
mudg_find=0
desv_find=sqrt(g_findtT_fr%*%VarDeltaX_find%*%t(g_findtT_fr))
VaRT_fr_find_DN[j]=(mudg_find)+(desv_find)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_find_DN[j]=-(mudg_find)-(desv_find)*dnorm(qnorm(alpha))/(1-alpha)


###VaR Cornish Fisher
tgammavar3_find=tgammavar2_find%*%(tgammavar_find)
asimP_find=sum(diag(tgammavar3_find))+3*g_findtT_fr%*%VarDeltaX_find%*%gamma_findtT2_fr%*%VarDeltaX_find%*%t(g_findtT_fr) #gamma tercer momento
asimP_find=asimP_find/(desv_find)^3  #coef de asimetría
 
VaRstd_find=qnorm(1-alpha)+((qnorm(1-alpha)^2-1)/6)*asimP_find  #VaR estandarizado

    
}
print("VaR total Delta Normal")
print(VaRT_find_DN)
print("CVaR total Delta Normal ")
print(CVaRT_find_DN)
print("VaR total Delta Normal por factor de riesgo")
print(VaRT_fr_find_DN)
print("CVaR total Delta Normal por factor de riesgo")
print(CVaRT_fr_find_DN)



[1] "VaR total Delta Normal"
          [,1]
[1,] -51116.04
[1] "CVaR total Delta Normal "
          [,1]
[1,] -60110.45
[1] "VaR total Delta Normal por factor de riesgo"
         [,1] [,2]      [,3]
[1,] -51013.7    0 -489.7291
[1] "CVaR total Delta Normal por factor de riesgo"
          [,1] [,2]      [,3]
[1,] -60133.64    0 -577.2802


####Total futuros


In [ ]:
%%R

DeltaX_fwdT = cbind(DeltaX_find,DeltaX_ftdc)
VarDeltaX_fwdT=as.matrix(cov(DeltaX_fwdT)*(n-1)/(n-2))


mu_fwdT=as.matrix(cbind(mu_ftdc,mu_find)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_fwdT=as.matrix(cbind(g_ftdctT,g_findtT)) #derivada con respecto las X's
#validar que ncol(X_port)=ncol(gt_port)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_fwdT=matrix(0,6,6)
gammat_fwdT[1:3,1:3]=gamma_ftdctT2
gammat_fwdT[4:6,4:6]=gamma_findtT2



#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_fwdT=(gammat_fwdT%*%VarDeltaX_fwdT)
tgammavar2_fwdT=(tgammavar_fwdT)%*%(tgammavar_fwdT) 
mu_fwdT=ifelse((sum(diag(tgammavar_fwdT))/2+sum(mu_fwdT))>0,0,(sum(diag(tgammavar_fwdT))/2+sum(mu_fwdT)))
desv_fwdT=(sqrt(gt_fwdT%*%VarDeltaX_fwdT%*%t(gt_fwdT)))
VaRT_fwdT_DN=mu_fwdT+desv_fwdT*qnorm(1-alpha) #VaR Normal
CVaRT_fwdT_DN=mu_fwdT-desv_fwdT*dnorm(qnorm(alpha))/(1-alpha)

print("VaR forward Total Delta Normal")
print(VaRT_fwdT_DN)
print("CVaR forward Total Delta Normal")
print(CVaRT_fwdT_DN)

[1] "VaR forward Total Delta Normal"
          [,1]
[1,] -33442.04
[1] "CVaR forward Total Delta Normal"
          [,1]
[1,] -39408.82


### Swaps

#### Por contrato

In [ ]:
%%R
DeltaXvp=Xvp[1:(n-1),]/Xvp[2:n,]-1
DeltaXtfwdt=XtfwdT[1:(n-1),]/XtfwdT[2:n,]-1

VaRcontDN_sw=matrix(0,1,ncol(N))
CVaRcontDN_sw=matrix(0,1,ncol(N))

V0f_dtt=dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dtc=dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dvp=dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
for (j in (1:ncol(N)))
{
  if(j==1)
  {
    mucont_sw=sum(V0f_dtt[1:sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc[1:sum(N[1:j])],V0f_dvp[1:sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,1:sum(N[1:j])],Xvp[1,1:sum(N[1:j])])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
  }
  else
  {
    mucont_sw=sum(V0f_dtt[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
    
  }
  
  
  
  #VaR Delta Normal
  VaRcontDN_sw[j]=sum(mucont_sw)+sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*qnorm(1-alpha)
  CVaRcontDN_sw[j]=sum(mucont_sw)+sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*dnorm(qnorm(1-alpha))/(1-alpha)
}

print("VaR por contrato Delta Normal")
print(VaRcontDN_sw)
print("CVaR por contrato Delta Normal")
print(CVaRcontDN_sw)

[1] "VaR por contrato Delta Normal"
          [,1]      [,2]
[1,] -42621.66 -15081.31
[1] "CVaR por contrato Delta Normal"
         [,1]     [,2]
[1,] 50249.14 17744.28


#### Por contrato factor de riesgo total

In [ ]:
%%R
WFR=rbind(c( XtfwdT[1,],Xvp[1,]*0),c( XtfwdT[1,]*0,Xvp[1,]))
VaRFrDN_sw=matrix(0,1,2)
CVaRFrDN_sw=matrix(0,1,2)

for (j in (1:2))
{
    
    g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]),
               dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))
    
    #Multiplicación de la derivada por el punto x0
    gt_sw=WFR[j,]*g_sw
    VaRFrDN_sw[j]=sqrt(gt_sw%*%VarDeltaX_sw%*%t(gt_sw))*qnorm(1-alpha)
    CVaRFrDN_sw[j]=-sqrt(gt_sw%*%VarDeltaX_sw%*%t(gt_sw))*dnorm(qnorm(1-alpha))/(1-alpha)
}

print("VaR por contrato fr total")
print(VaRFrDN_sw)
print("CVaR por contrato fr total")
print(CVaRFrDN_sw)

[1] "VaR por contrato fr total"
          [,1]     [,2]
[1,] -55827.97 -45.7393
[1] "CVaR por contrato fr total"
          [,1]      [,2]
[1,] -65808.59 -53.91632


#### Contrato Factor de riesgo marginal

In [ ]:
%%R
VaRcontfrDN_sw=matrix(0,2,ncol(N))
CVaRcontfrDN_sw=matrix(0,2,ncol(N))
WFRcont <- rbind(c(1,0), c(0,1))

V0f_dtt=dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dtc=dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
V0f_dvp=dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,])
for (j in (1:ncol(N)))
{
    for(k in (1:2)){
  if(j==1)
  {
    mucont_sw=sum(V0f_dtt[1:sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc[1:sum(N[1:j])],V0f_dvp[1:sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,1:sum(N[1:j])]*WFRcont[k,1],Xvp[1,1:sum(N[1:j])]*WFRcont[k,2])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,1:sum(N[1:j])],DeltaXvp[,1:sum(N[1:j])]))*(n/(n-1))	
    }
  else
  {
    mucont_sw=sum(V0f_dtt[sum(N[1:j-1]+1):sum(N[1:j])])
    gcont_sw=as.matrix(c(V0f_dtc[sum(N[1:j-1]+1):sum(N[1:j])],V0f_dvp[sum(N[1:j-1]+1):sum(N[1:j])]))
    gtcont_sw=t(as.matrix(gcont_sw*c(XtfwdT[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont[k,1],Xvp[1,sum(N[1:j-1]+1):sum(N[1:j])]*WFRcont[k,2])))
    VarCovXcont_sw=cov(cbind(DeltaXtfwdt[,sum(N[1:j-1]+1):sum(N[1:j])],DeltaXvp[,sum(N[1:j-1]+1):sum(N[1:j])]))*(n/(n-1))
    }
  
  
  #VaR Delta Normal
  VaRcontfrDN_sw[k,j]=sum(mucont_sw)+sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*qnorm(1-alpha)
  CVaRcontfrDN_sw[k,j]=sum(mucont_sw)-sqrt(gtcont_sw%*%VarCovXcont_sw%*%t(gtcont_sw))*dnorm(qnorm(1-alpha))/(1-alpha)
}
}

print("VaR por contrato fr Delta Normal")
print(VaRcontfrDN_sw)
print("CVaR por contrato fr Delta Normal")
print(CVaRcontfrDN_sw)

[1] "VaR por contrato fr Delta Normal"
             [,1]         [,2]
[1,] -42553.16166 -15126.08903
[2,]    -88.46422    -66.63725
[1] "CVaR por contrato fr Delta Normal"
            [,1]         [,2]
[1,] -50161.2224 -17827.52483
[2,]   -104.9209    -75.82748


#### Total

In [ ]:
%%R
#Evalución de las derivada en el punto x0

g_sw=cbind(dswap_dtc(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]),
        dswap_dvp(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))

#Multiplicación de la derivada por el punto x0
gt_sw=c( XtfwdT[1,],Xvp[1,])*g_sw
VaRTotalDN_sw=sqrt(gt_sw%*%VarDeltaX_sw%*%t(gt_sw))*qnorm(1-alpha)+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))
CVaRTotalDN_sw = -sqrt(gt_sw%*%VarDeltaX_sw%*%t(gt_sw))*dnorm(qnorm(1-alpha))/(1-alpha)+sum(dswap_dtt(por_swT, contratos_swT, nominal_swT, XtfwdT[1,], tasafija_swT,  plazocupon_swT, VTplazos_sw, Xvp[1,]))


print("VaR Total Delta Normal")
print(VaRTotalDN_sw)
print("CVaR Total Delta Normal")
print(CVaRTotalDN_sw)


[1] "VaR Total Delta Normal"
          [,1]
[1,] -55864.55
[1] "CVaR Total Delta Normal"
          [,1]
[1,] -65849.62


### Opciones

####Riesgo a nivel contrato y nivel contrato-factor de riesgo

In [ ]:
%%R
print("VaR a nivel contrato Delta Normal")
print(VaRmargcDN_oir)
print("CVaR a nivel contrato Delta Normal")
print(CVaRmargcDN_oir)

print("VaR a nivel contrato factor de riesgo delta normal")
print(VaRmargcDN_fr_oir)
print("CVaR a nivel contrato factor de riesgo delta normal")
print(CVaRmargcDN_fr_oir)


[1] "VaR a nivel contrato Delta Normal"
            [,1]        [,2]
[1,] -0.08757515 -0.04730174
[1] "CVaR a nivel contrato Delta Normal"
           [,1]        [,2]
[1,] -0.1032314 -0.05575808
[1] "VaR a nivel contrato factor de riesgo delta normal"
            [,1]          [,2]
[1,] -0.01154790 -0.0009612335
[2,] -0.06765790 -0.0365505436
[3,] -0.05238478 -0.0327790004
[1] "CVaR a nivel contrato factor de riesgo delta normal"
            [,1]         [,2]
[1,] -0.01361237 -0.001133077
[2,] -0.07975340 -0.043084848
[3,] -0.06174984 -0.038639049


####Riesgo a nivel total y total factor de riesgo

In [ ]:
%%R
#RIESGO TOTAL FORWARDS DE TIPO DE CAMBIO
m=ncol(plazos_oir)
VarDeltaX_oir=cov(DeltaX_oir)*(n/(n-1))

tgammavar_oir=(gammaT_oirT2%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
mudg_oir=ifelse(sum(mu_oir)>0,0,sum(mu_oir))
desv_oir=sqrt(deltaT_oir_T%*%VarDeltaX_oir%*%t(deltaT_oir_T))

VaRT_oir_DN=(mudg_oir)+(desv_oir)*qnorm(1-alpha)  #VaR Normal
CVaRT_oir_DN=(mudg_oir)-(desv_oir)*dnorm(qnorm(alpha))/(1-alpha)

print("VaR Total Delta Normal")
print(VaRT_oir_DN)
print("CVaR Total Delta Normal")
print(CVaRT_oir_DN)


VaRT_fr_oir_DN=matrix(0,1,3)
CVaRT_fr_oir_DN=matrix(0,1,3)


WFRT=matrix(0,3,(m*3))


#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRM=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRM[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)  #PASO CLAVE    
gamma_oirtT2_fr=gammaT_oirT2*WFRM #evaluación de gamma multiplicado por x^{2}
g_oirtT_fr=deltaT_oir_T*WFRT[j,]
tgammavar_oir=(gamma_oirtT2_fr%*%VarDeltaX_oir)
tgammavar2_oir=(tgammavar_oir)%*%(tgammavar_oir) 
#mudg_oir=sum(diag(tgammavar_oir))/2+sum(mu_oir)
mudg_oir=0
desv_oir=sqrt(g_oirtT_fr%*%VarDeltaX_oir%*%t(g_oirtT_fr))
VaRT_fr_oir_DN[j]=(mudg_oir)+(desv_oir)*qnorm(1-alpha) #VaR Normal
CVaRT_fr_oir_DN[j]=(mudg_oir)-(desv_oir)*dnorm(qnorm(alpha))/(1-alpha)


}


print("VaR por contrato fr total")
print(VaRT_fr_oir_DN)
print("CVaR por contrato fr total")
print(CVaRT_fr_oir_DN)


[1] "VaR Total Delta Normal"
          [,1]
[1,] -0.118298
[1] "CVaR Total Delta Normal"
           [,1]
[1,] -0.1394467
[1] "VaR por contrato fr total"
            [,1]        [,2]        [,3]
[1,] -0.01243328 -0.08778916 -0.07886964
[1] "CVaR por contrato fr total"
            [,1]       [,2]        [,3]
[1,] -0.01465603 -0.1034836 -0.09296952


### Riesgo integral

##### Total

In [ ]:
%%R
VarDeltaX_port=as.matrix(cov(DeltaX_port)*(n-1)/(n-2))

mu_port=as.matrix(cbind(mu_acc_div,mu_bcc,mu_bm,mu_bd,mu_ftdc,mu_find,mu_sw,mu_oir)) #derivada con respecto al tiempo
#INTEGRACIÓN DE PRIMERAS DERIVADAS (GRADIENTE)
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T)) #derivada con respecto las X's
#validar que ncol(X_port)=ncol(gt_port)


#INTEGRACIÓN DE MATRIZ HESSIANA

gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2



#CÁLCULO DE VAR Y CVAR A NIVEL PORTAFOLIO
tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_DN=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_DN=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)


print(VaRT_port_DN)
print(CVaRT_port_DN)




          [,1]
[1,] -76468.43
          [,1]
[1,] -90130.61


#### Acciones

In [ ]:
%%R
m = ncol(plazos_fwd_ind)
WFRT_acc=matrix(0,3,(m*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT_acc[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRT_acc=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRT_acc[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)}

dummy_acc = matrix(0,1, ncol(gt_port))
dummy_acc[(m_fx+1):(m_fx+m_acc)] = 1
dummy_acc[c(168)] = 1 #Cambia dependiendo del factor de riesgo a tomar. Revisar los riesgos de las opciones IPC
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T))

gt_port = gt_port*dummy_acc
gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2

gammat_port = gammat_port * t(dummy_acc)%*%dummy_acc

tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_acc_DN=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_acc_DN=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)




print(VaRT_port_acc_DN)
print(CVaRT_port_acc_DN)


          [,1]
[1,] -52688.68
          [,1]
[1,] -62099.28


#### Divisas

In [ ]:
%%R
m = ncol(plazos_fwd)
WFRT_div=matrix(0,3,(m*3))

#RIESGO TOTAL POR FACTOR DE RIESGO DE FORWARDS DE TIPO DE CAMBIO

for (j in (1:3))
{
WFRT_div[j,((j-1)*m+1):(j*m)]=matrix(1,1,m)    #Matriz de diseño para 
WFRT_div=matrix(0,(m*3),(m*3))  #PASO CLAVE
WFRT_div[((j-1)*m+1):(m*j),((j-1)*m+1):(m*j)]=matrix(1,m,m)}

dummy_div = matrix(0,1, ncol(gt_port))
dummy_div[1:m_fx] = 1
dummy_div[c(165, 166, 167)] = 1 #Cambia dependiendo del factor de riesgo a tomar.
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T)) #derivada con respecto las X's

gt_port = gt_port*dummy_div
gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2

gammat_port = gammat_port * t(dummy_div)%*%dummy_div

tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_div_DN=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_div_DN=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)



print(VaRT_port_div_DN)
print(CVaRT_port_div_DN)


          [,1]
[1,] -449.7078
          [,1]
[1,] -521.3069


#### Tasa interés

In [ ]:
%%R

dummy_tint = matrix(0,1, ncol(gt_port))
dummy_tint[7:sum(n_if[2:3], n_if[4])] = 1
dummy_tint[c(167, 170, 172)] = 1
dummy_tint[sum(n_if[1:7],1):length(dummy_tint)] = 1 #Cambia dependiendo del factor de riesgo a tomar.
gt_port=as.matrix(cbind(g_acc_divt,g_bcct,g_bmt,g_bdt,g_ftdctT,g_findtT,gt_sw,deltaT_oir_T)) #derivada con respecto las X's

gt_port = gt_port*dummy_tint
gammat_port=matrix(0,sum(n_if),sum(n_if))
gammat_port[1:n_if[1],1:n_if[1]]=gamma_acc_divt
gammat_port[(sum(n_if[1:1])+1):(sum(n_if[1:2])),(sum(n_if[1:1])+1):(sum(n_if[1:2]))]=gamma_bcct2
gammat_port[(sum(n_if[1:2])+1):(sum(n_if[1:3])),(sum(n_if[1:2])+1):(sum(n_if[1:3]))]=gamma_bmt2
gammat_port[(sum(n_if[1:3])+1):(sum(n_if[1:4])),(sum(n_if[1:3])+1):(sum(n_if[1:4]))]=gamma_bdt2
gammat_port[(sum(n_if[1:4])+1):(sum(n_if[1:5])),(sum(n_if[1:4])+1):(sum(n_if[1:5]))]=gamma_ftdctT2
gammat_port[(sum(n_if[1:5])+1):(sum(n_if[1:6])),(sum(n_if[1:5])+1):(sum(n_if[1:6]))]=gamma_findtT2
gammat_port[(sum(n_if[1:6])+1):(sum(n_if[1:7])),(sum(n_if[1:6])+1):(sum(n_if[1:7]))]=tgamma_sw
gammat_port[(sum(n_if[1:7])+1):(sum(n_if[1:8])),(sum(n_if[1:7])+1):(sum(n_if[1:8]))]=gammaT_oirT2

gammat_port = gammat_port * t(dummy_tint)%*%dummy_tint

tgammavar_port=(gammat_port%*%VarDeltaX_port)
tgammavar2_port=(tgammavar_port)%*%(tgammavar_port) 
mu_portT=ifelse((sum(diag(tgammavar_port))/2+sum(mu_port))>0,0,(sum(diag(tgammavar_port))/2+sum(mu_port)))
desv_portT=(sqrt(gt_port%*%VarDeltaX_port%*%t(gt_port)))
VaRT_port_tint_DN=mu_portT+desv_portT*qnorm(1-alpha) #VaR Normal
CVaRT_port_tint_DN=mu_portT-desv_portT*dnorm(qnorm(alpha))/(1-alpha)




print(VaRT_port_tint_DN)
print(CVaRT_port_tint_DN)


          [,1]
[1,] -2565.447
          [,1]
[1,] -3015.656
